In [ ]:
#From https://github.com/gabrielo/Allegheny-County-Property-Assessments

In [394]:
# assessments/cd/AC Property Assessments_10012017.xls (which is actually TDF) from Amy Gottsegen and Randy Sargent buying a CD from the county assessors office
# assessments/Allegheny_County_Parcel_Boundaries.geojson from https://data.wprdc.org/dataset/allegheny-county-parcel-boundaries
import array, csv, datetime, json, math, numpy, os, random, re 
from dateutil.parser import parse
from shapely.geometry import mapping, shape
from PIL import Image
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import pandas,numbers
from operator import itemgetter, attrgetter
import string
import calendar,time
from IPython.core.display import HTML

# Use geopandas.  Boilerplate from https://docs.google.com/document/d/1utZuLHcKQEZNXTQLOysTNCxTHrqxczAUymmtplpn27Q/edit#heading=h.f50xoxwmcir
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
%matplotlib inline

# Built-in time handling fails for times before 1900.  Use arrow instead.  
# See https://arrow.readthedocs.io/en/latest/ for info
import arrow

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]


def GetCentroid(geometry):
    s = shape(geometry)
    return (s.centroid.x, s.centroid.y)   

def GetEpoch(date):
    return (date - datetime.datetime(1970, 1, 1)).total_seconds()

def HexToRgb(hex_string):
    rgb = colors.hex2color(hex_string)
    r,g,b = tuple([int(255*x) for x in rgb])
    return (r,g,b)

# Generate list of residental PARIDs

In [3]:
# Read in 2017 data to harvest list of residential parcel IDs
# We need to do this before loading the older ADB files because they're missing some of the columns we need.

In [100]:
# Read in assessments data as TDF
path = "assessments/cd/AC Property Assessments_10012017.xls"
apd_2017 = pandas.read_csv(path,sep='\t',dtype={'PROPERTYHOUSENUM':numpy.str,'CHANGENOTICEADDRESS4':numpy.str, 'PROPERTYZIP':numpy.str})

/home/rsargent/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (20,21,30,31,32,39,48,63,89) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# Return an array of PARID values matching either residential or apartments (usedesc includes APART)
def get_residential_parids(apd):    
    res_parids=[]
    for i in range(0, len(apd.index)):
        par_id = apd['PARID'][i]
        class_description = apd['CLASSDESC'][i] 
        usedesc = apd['USEDESC'][i]

        if(class_description != 'RESIDENTIAL' and (isinstance(usedesc, numbers.Number) or 'APART' not in usedesc)):
            # skip this one
            continue
        res_parids.append(par_id)
    return(res_parids)

In [8]:
res_parids=get_residential_parids(apd_2017)
len(res_parids)

519728

In [99]:
apd_2017

,PARID,PROPERTYOWNER,PROPERTYHOUSENUM,PROPERTYFRACTION,PROPERTYADDRESS,PROPERTYCITY,PROPERTYSTATE,PROPERTYUNIT,PROPERTYZIP,MUNICODE,MUNIDESC,SCHOOLCODE,SCHOOLDESC,LEGAL1,LEGAL2,LEGAL3,NEIGHCODE,NEIGHDESC,TAXCODE,TAXDESC,TAXSUBCODE,TAXSUBCODE_DESC,OWNERCODE,OWNERDESC,CLASS,CLASSDESC,USECODE,USEDESC,LOTAREA,HOMESTEADFLAG,FARMSTEADFLAG,CLEANGREEN,ABATEMENTFLAG,RECORDDATE,SALEDATE,SALEPRICE,SALECODE,SALEDESC,DEEDBOOK,DEEDPAGE,PREVSALEDATE,PREVSALEPRICE,PREVSALEDATE2,PREVSALEPRICE2,AGENT,TAXFULLADDRESS1,TAXFULLADDRESS2,TAXFULLADDRESS3,TAXFULLADDRESS4,CHANGENOTICEADDRESS1,CHANGENOTICEADDRESS2,CHANGENOTICEADDRESS3,CHANGENOTICEADDRESS4,COUNTYBUILDING,COUNTYLAND,COUNTYTOTAL,COUNTYEXEMPTBLDG,LOCALBUILDING,LOCALLAND,LOCALTOTAL,FAIRMARKETBUILDING,FAIRMARKETLAND,FAIRMARKETTOTAL,STYLE,STYLEDESC,STORIES,YEARBLT,EXTERIORFINISH,EXTFINISH_DESC,ROOF,ROOFDESC,BASEMENT,BASEMENTDESC,GRADE,GRADEDESC,CONDITION,CONDITIONDESC,CDU,CDUDESC,TOTALROOMS,BEDROOMS,FULLBATHS,HALFBATHS,HEATINGCOOLING,HEATINGCOOLINGDESC,FIREPLACES,BSMTGARAGE,FINISHEDLIVINGAREA,CARDNUMBER,ALT_ID,TAXYEAR,ASOFDATE
0,0001H00303000000,YASEEN BUSHRA,409,,SMITHFIELD ST,PITTSBURGH,PA,,15222,101,1st Ward - PITTSBURGH,47,City Of Pittsburgh,LOT 20.31XAVG60.20X20.09,NaN,NaN,51C01,PITTSBURGH URBAN,T,20 - Taxable,NaN,NaN,10,REGULAR,C,COMMERCIAL,406,RETL/STOR OVER,1600,NaN,NaN,NaN,NaN,03-26-2013,03-26-2013,120000.0,Z,OTHER,15187,437,12-21-2007,120000.0,09-04-1998,150000.0,YASEEN BUSHRA,3510 MEADOWGATE DR,,MURRYSVILLE PA,15668,3510 MEADOWGATE DR,,MURRYSVILLE PA,15668,70900,96000,166900,0,70900,96000,166900,70900,96000,166900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,01-OCT-17
1,0001H00306000000,PELE CAPITAL LLCBEYNON & CO,345,-347,4TH AVE,PITTSBURGH,PA,,15222,101,1st Ward - PITTSBURGH,47,City Of Pittsburgh,LOT 60.22X60.34-4TH AVE,12 STY GRANITE & BRK BANK & OFF BLDG 345-347,NaN,51C01,PITTSBURGH URBAN,T,20 - Taxable,NaN,NaN,20,CORPORATION,C,COMMERCIAL,404,RETL/APT'S OVER,3600,NaN,NaN,NaN,Y,01-20-2015,12-29-2014,5000000.0,DT,DATE OF TRANSFER,15860,287,09-05-2013,2000150.0,05-02-1997,507500.0,BEYNON & CO,429 FORBES AVE STE 1900,,PITTSBURGH PA,15219,429 FORBES AVE,1900 ALLEGHENY AVE,PITTSBURGH PA,15219,1252400,432000,1684400,747300,1999700,432000,2431700,1999700,432000,2431700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,01-OCT-17
2,0001H00309000000,341 FOURTH AVENUE LLC,341,,4TH AVE,PITTSBURGH,PA,,15222,101,1st Ward - PITTSBURGH,47,City Of Pittsburgh,LOT 45.22X120.67 FOURTH AV BET WOOD & SMITHFIEL,D STS,7 STY BRK OFF BLDG #341-343,51C01,PITTSBURGH URBAN,T,20 - Taxable,NaN,NaN,20,CORPORATION,C,COMMERCIAL,404,RETL/APT'S OVER,10890,NaN,NaN,NaN,NaN,12-26-2002,12-26-2002,500000.0,XX,OTHER,11536,326,11-07-2001,250000.0,11-07-2001,1.0,"341 FOURTH AVENUE, LLC",435 DONNER AVE STE 520,,MONESSEN PA,15062,435 DONNER AVE STE 520,,MONESSEN PA,15062,1743000,653400,2396400,0,1743000,653400,2396400,1743000,653400,2396400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,01-OCT-17
3,0001H00311000000,ESWP LLC,337,,4TH AVE,PITTSBURGH,PA,,15222,101,1st Ward - PITTSBURGH,47,City Of Pittsburgh,LOT 45.49X120.67,NaN,NaN,51C01,PITTSBURGH URBAN,T,20 - Taxable,NaN,NaN,20,CORPORATION,C,COMMERCIAL,465,LODGE HALL/AMUSEMENT PARK,11132,NaN,NaN,NaN,NaN,11-29-2004,11-29-2004,3.0,3,LOVE&AFFECTION,12272,37,08-13-1987,767000.0,NaN,NaN,"ESWP, LLC",337 4TH AVE,,PITTSBURGH PA,15222,337 4TH AVE,,PITTSBURGH PA,15222,628100,667900,1296000,0,628100,667900,1296000,628100,667900,1296000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,01-OCT-17
4,0001H00324000000,BANK TOWER REALTY LLC,307,,4TH AVE,PITTSBURGH,PA,,15222,101,1st Ward - PITTSBURGH,47,City Of Pittsburgh,LOT 80.11X120.92X80.3 FOURTH AVE,3/15 STY BRK STL STN GRANITE BLDG,NaN,51C01,PITTSBURGH URBAN,T,20 - Taxable,NaN,NaN,20

# Create structures to populate with sale and ownership information

In [9]:
# For each residential PARID, we want to generate a sorted list of who bought it when.
# For each owner name, we want to generate a sorted list of PARIDs they bought and when they bought it.
# For each change address, we want to generate a sorted list of PARIDs they bought and when they bought it.
# Each of the following is a map indexed on PARID, owner name, or change address.  

# The map indexed on PARID builds up a history of a given property.
#   Each record contains a a list of maps containing a list of date, event_type, owner name, and change address
#   event_type is either PURCHASE or FORECLOSURE

# Each map indexed on owner name or change address builds a history of a given owner/change address.
#   Each record contains a list of maps containing date, event_type, and PARID
#   event_type is either PURCHASE or SALE


In [206]:
# To start with, clear the maps
property_map = {}
owner_map = {}
changeaddr_map = {}

In [385]:
# Create functions to update the above maps with info from a given row of an assessment spreadsheet/database.  
# The first argument should be a pandas table, the second an index into the table.
# We expect the following columns to exist and be valid:
#   PIN
#   PROPERTYOWNER
#   CHANGENOTICEADDRESS1 - CHANGENOTICEADDRESS4
#   SALEDATE


# In 2017 the date format is MM-DD-YYYY
# In 2009, an example is 09/16/96 00:00:00

# Setup regular expression for parsing various sorts of dates
re_MDY = re.compile('(\d\d)-(\d\d)-(\d\d\d\d)')
re_YMD = re.compile('(\d\d\d\d)-(\d\d)-(\d\d)')
re_MDY_HMS = re.compile('(\d+)/(\d+)/(\d+) (\d+):(\d+):(\d+)')

# Setup this year to use for dealing with 2-digit dates
this_year = datetime.datetime.now().year
this_date = arrow.now().format('YYYY-MM-DD')

# Utility function to normalize SALEDATE format
# SALEDATE field is in '%m-%d-%Y' format (ex 10-26-2012) when present
def SaledateToEpoch(datestr):
    if(re_YMD.match(datestr)!=None):
        return calendar.timegm(time.strptime(datestr, '%Y-%m-%d'))
    elif(re_MDY.match(datestr)!=None):
        return calendar.timegm(time.strptime(datestr, '%m-%d-%Y'))

# Built-in time handling fails for times before 1900.  Use arrow instead.  
# See https://arrow.readthedocs.io/en/latest/ for info
def SaledateToYMD(datestr):
    # Check which format and parse accordingly
    if(re_YMD.match(datestr)!=None):
        # Already like we want it
        return datestr
    if(re_MDY.match(datestr)!=None):
        return arrow.get(datestr, 'MM-DD-YYYY').format('YYYY-MM-DD')
    elif(re_MDY_HMS.match(datestr)!=None):
        # Some of the 2-digit years are farther back than python's default pivot year of 1969
        # If the year we parse is > this_year, set it back by 100 years
        ad=arrow.get(datestr, 'MM/DD/YY HH:mm:ss')
        if(ad.year>this_year):
            ad = ad.replace(year=(ad.year-100))
        return ad.format('YYYY-MM-DD')
    else:
        raise Exception('Unrecognized saledate format %r' % (datestr))

# find_event_by_date takes a list of maps containng 'date' fields and looks for one the the last one that's before
# search_date or the first which is simultaneous (in case of multiple simultaneous records).  
# Returns -1 if the search_date is earlier than any existing item.  It assumes that each map in the event list 
# contains a 'date' field and that they're in sorted order from earliest to latest.
def find_event_index_by_date(event_list, search_date):
    match_index=-1
    for i in range(0,len(event_list)):
        if(event_list[i]['date']<search_date):
            # This happened before the date we're looking for, update match_index
            match_index=i
        elif(event_list[i]['date']==search_date):
            # This is the first exact match, return i
            return(i)
        else:
            # This happened after the date we're looking for, return match_index
            return match_index
    # If we get to here, there is no event after this date.  Return match_index
    return match_index

# This takes an existing event list and either inserts a given event_map and returns True, or 
# decides that the event is a duplicate and returns false
def insert_event(event_list, event_map):
    # Find the index of the element already in the list which precedes the date in event_map.  
    # If not found, match_index is -1, so we should insert it at the beginning of event_list and return True
    # If found, check if the record matches what we already have (TODO: see if we need something
    # more sophisticated than ==).  If it matches what we already have, return False and do not modify the list.
    # If it doesn't match what we have insert it.  Note that list.insert takes the arg of the element to insert
    # before, so we use match_index+1
    match_index = find_event_index_by_date(event_list, event_map['date'])
    if(match_index==-1):
        #print "No match, insert at beginning"
        # No match.  Insert at the beginning, return True
        event_list.insert(0, event_map)
        return True
    # We've got a matching or preceeding event, compare it with this and any other potentially 
    # simultaneous events
    #print "Time match at %d" % (match_index)
    for i in range(match_index, len(event_list)):
        if(event_map == event_list[i]):
            # This is a duplicate, ignore it
            #print "Found duplicate match at %d (%r)" % (i,event_list[i])
            return False
        elif(event_map['date']<event_list[i]['date']):
            # We're past any simultaneous events that might have matched, continue
            #print "Hit a later item at %d (%r)" % (i,match_index)
            break
        #print "Didn't match at %d (%r), keep trying" % (i,event_list[i])
        match_index=i
    # We've got a preceeding or simultaneous event that doesnt match.  Insert this 
    # new item just past match_index
    #print "Inserting after %d" % (match_index)
    event_list.insert(match_index+1, event_map)
    return True
        
# Clean up multiple whitespace and/or <BR> to a single space
def cleanup_str(str_in):
    str_out = re.sub( '<BR>', ' & ', str_in)
    str_out = re.sub( '\s+', ' ', str_out).strip()
    return(str_out)

# Try to normalize changeaddr to be more likely to match
re_zp4 = re.compile('(.+) (\d\d\d\d\d)-(\d\d\d\d)?')
re_cscz = re.compile('(.+) (\w\w), (\d\d\d\d\d)')
# Check for change address with no alphanumerics in it
re_noalnum = re.compile('^([^\w]+)$')
def get_owner_changeaddr(apd, i):
    c_list = []
    if(not (pandas.isnull(apd['CHANGENOTICEADDRESS1'][i]) or apd['CHANGENOTICEADDRESS1'][i]=='')):
        c = string.strip(str(apd['CHANGENOTICEADDRESS1'][i]))
        if(c!=''):
            c_list.append(c)
    if(not (pandas.isnull(apd['CHANGENOTICEADDRESS2'][i]) or apd['CHANGENOTICEADDRESS2'][i]=='')):
        c = string.strip(str(apd['CHANGENOTICEADDRESS2'][i]))
        if(c!=''):
            c_list.append(c)
    if(not (pandas.isnull(apd['CHANGENOTICEADDRESS3'][i]) or apd['CHANGENOTICEADDRESS3'][i]=='')):
        c = string.strip(str(apd['CHANGENOTICEADDRESS3'][i]))
        if(c!=''):
            c_list.append(c)
    if(not (pandas.isnull(apd['CHANGENOTICEADDRESS4'][i]) or apd['CHANGENOTICEADDRESS4'][i]=='')):
        c = string.strip(str(apd['CHANGENOTICEADDRESS4'][i]))
        if(c!=''):
            c_list.append(c)
    ret_str = cleanup_str(", ".join(c_list))
    
    # Check to make sure it's got at least one alphanumeric
    m = re_noalnum.match(ret_str)
    if(m!=None):
        print "Skipping invalid changeaddr %r" % (ret_str)
        return ''
    
    # Check for zip+4, and if so cut it down to regular zip.  2009 didn't have zip+4
    m = re_zp4.match(ret_str)
    if(m!=None):
        #print "Getting rid of zip+4 %r" %(ret_str)
        ret_str = "%s %s" % (m.group(1),m.group(2))
        #print "  Result = %r" %(ret_str)
        
    # Check for city state, zip.  If so, replace with city, state zip
    m = re_cscz.match(ret_str)
    if(m!=None):
        #print "Getting rid of city state, zip %r" %(ret_str)
        ret_str = "%s, %s %s" % (m.group(1),m.group(2),m.group(3))
        #print "  Result = %r" %(ret_str)
        
    # Check for PGH, change to PITTSBURGH
    ret_str = re.sub( ' PGH,', ' PITTSBURGH,', ret_str)
    
    return(ret_str)

# First arg is a pandas table, second is an index for what to process.
# This updates property_map, owner_map, and changeaddr_map appropriately
# Returns True on success, False on failure
def process_assessment_record(apd, i):
    # Extract PARID, saledate, owner_name, and changeaddr from the record
    par_id = apd['PARID'][i]
    
    # Make sure it's a residential record.  If this fails, make sure to run get_residential_parids on 
    # a dataset (like apd_2017) that has the full set of columns before running this function
    if(not par_id in res_parids):
        return

    # Process saledate
    saledate_raw = apd['SALEDATE'][i]
    # If saledate is valid, keep it.  Otherwise return False
    if(saledate_raw == '' or (isinstance(saledate_raw, numbers.Number) and math.isnan(saledate_raw))):
        return False
    
    # Convert saledatet YMD so it sorts properly 
    saledate = SaledateToYMD(saledate_raw)
    
    owner_name = cleanup_str(apd['PROPERTYOWNER'][i])
    owner_changeaddr = get_owner_changeaddr(apd,i)
        
    if(owner_changeaddr == ''):
        # skip this one
        return False
    
    # We have valid par_id, owner_name, and owner_changeaddr.  Update the maps.
    property_event={'date': saledate, 'event_type':'PURCHASE', 'ownername':owner_name, 'changeaddr': owner_changeaddr}
    owner_event={'date':saledate, 'event_type':'PURCHASE', 'parid':par_id}

    #print "-------\nProcessing %s: %r %r %r" % (par_id, saledate, owner_name, owner_changeaddr)

    if(not property_map.has_key(par_id)):
        # We don't have an entry for this property yet, create an empty list
        property_map[par_id]=[]
    
    # Insert the property event
    pret = insert_event(property_map[par_id], property_event)
 
    if(not owner_map.has_key(owner_name)):
        # We don't have an entry for this owner name yet, create an empty list
        owner_map[owner_name]=[]
    
    # Insert the owner event to owner map
    oret = insert_event(owner_map[owner_name], owner_event)
   
    if(not changeaddr_map.has_key(owner_changeaddr)):
        # We don't have an entry for this changeaddr yet, create an empty list
        changeaddr_map[owner_changeaddr]=[]
    
    # Insert the owner event
    cret = insert_event(changeaddr_map[owner_changeaddr], owner_event)
    
    # Print diagnostics
    #print "%s: %r %r %r = %r %r %r" % (par_id, saledate, owner_name, owner_changeaddr, pret, oret, cret)
    return(pret or oret or cret)


In [333]:
this_date

u'2018-08-08'

In [166]:
cleanup_str("a   <BR>   b\t\nc")

'a b c'

In [386]:
SaledateToEpoch('08-08-2012')

1344384000

In [387]:
SaledateToEpoch('2012-08-08')

1344384000

In [230]:
SaledateToYMD('08-08-2012')

u'2012-08-08'

In [231]:
SaledateToYMD('1993-12-20')

'1993-12-20'

In [232]:
SaledateToYMD('09/16/96 00:00:00')

u'1996-09-16'

In [233]:
SaledateToYMD('09/16/50 00:00:00')

u'1950-09-16'

In [207]:
# Test with entries from apd_2017
for i in range(0,len(apd_2017)):
    process_assessment_record(apd_2017,i)

0001H00327020100: '2014-06-02' 'SNYDER JOHN J & JULIE A (W)' '513 BROUGHTON RD, BETHEL PARK, PA 15102' = True True True
0001H00327020200: '2012-08-06' 'WRIGHT DEREK EDWARD' '306 4TH AVE UNIT 202, PITTSBURGH, PA 15222' = True True True
0001H00327030100: '2009-10-07' 'OCONNOR THOMAS C & RUBAB JAFRY (W)' '300 4TH AVE UNIT 301, PITTSBURGH, PA 15222' = True True True
0001H00327030200: '2009-11-02' 'LESKER KURT J IV' '300 4TH AVE UNIT 302, PITTSBURGH, PA 15222' = True True True
0001H00327040100: '2010-11-04' 'ROLAND DAVID BRUCE & ELIZABETH CARNEAL(W)' '300 4TH AVE UNIT 401, PITTSBURGH, PA 15222' = True True True
0001H00327040200: '2010-02-12' 'WALDBAUM HEIDE B' '300 4TH AVE UNIT 402, PITTSBURGH, PA 15222' = True True True
0001H00327040300: '2015-04-24' 'BURKLECA LEE FRANK & RACHEL (W)' '300 4TH AVE STE 403, PITTSBURGH, PA 15222' = True True True
0001H00327050100: '2015-11-17' 'MCGROGAN HALLER MARIAH' '300 FOURTH AVE 501, PITTSBURGH, PA 15222' = True True True
0001H00327050200: '2013-06-06' '

0001H00327180300: '2016-09-01' 'YANNUZZI JEFFREY V' '2108 W GROVE DR, GIBSONIA, PA 15044' = True True True
0001H00327190100: '2013-02-15' 'GARESCHE CARL E' '300 4TH AVE UNIT 1901, PITTSBURGH, PA 15222' = True True True
0001H00327200100: '2016-12-22' 'FOSTER GARY M' '300 4TH AVE UNIT 2001, PITTSBURGH, PA 15222' = True True True
0001H00327200200: '2015-05-18' 'FREDRIKSEN KATHARINE' '300 4TH AVE UNIT 2002, PITTSBURGH, PA 15222' = True True True
0001H00327200300: '2009-12-31' 'JAMESON ROBERT C & LINDA M (W)' '10049 HAMBELTON ST, LIVONIA, MI 48150' = True True True
0001H00370000000: '2009-09-02' 'RESIDENCES AT WOOD STREET' '425 6TH AVE STE 950, C/O ACTION HOUSING, PITTSBURGH, PA 15219' = True True True
0001G00224160200: '2013-08-29' 'NALEPPA PATRICIA A' '151 FORT PITT BLVD UNIT 1602, PITTSBURGH, PA 15222' = True True True
0001G00224160300: '2014-09-12' 'FALBO RALPH A JR & MARGARET J (W)' '151 FORT PITT BLVD UNIT 1603, PITTSBURGH, PA 15222' = True True True
0001G00224160400: '2017-04-26' 'MF

0011J00072000A00: '2016-06-09' 'PITTSBURGH PROPERTY RENTALS L P' '736 W INGOMAR RD, INGOMAR, PA 15127' = True True True
0011J00072000C00: '1981-04-03' 'BRONSON SALLY ANN' '1729 FORBES AVE, PITTSBURGH, PA 15219' = True True True
0011J00072000D00: '1989-10-13' 'BRONSON SALLY & ANGELA M BRONSON' '1729 FORBES AVE, PITTSBURGH, PA 15219' = True True True
0011J00072000E00: '1989-01-19' 'TONEY JOSEPH G & HELEN G AIMAN' '1731 FORBES AVE, PITTSBURGH, PA 15219' = True True True
0011J00072000F00: '2010-11-22' 'BRGS LLC' 'PO BOX 13444, PITTSBURGH, PA 15243' = True True True
0011J00073000A00: '1999-06-07' 'FENICOTO SALAVATORE & GIROLAMA (W)' '1708 WATSON ST, PITTSBURGH, PA 15219' = True True True
0011J00073000B00: '1995-06-30' 'FENICATO SALVATORE & GIROLAMA (W)' '1708 WATSON ST, PITTSBURGH, PA 15219' = True True True
0011J00074000000: '2010-11-22' 'BRGS LLC' 'PO BOX 13444, PITTSBURGH, PA 15243' = True True True
0011J00074000B00: '2002-10-31' 'NEWKIRK THOMAS' '1723 FORBES AVE, PITTSBURGH, PA 15219' =

0001C01662023700: '2006-03-23' 'FRIEDLANDER SUMMER G' '1 BROOKMERE DR, GREENSBURG, PA 15601' = True True True
0001C01662024200: '2015-09-04' 'HATTLER JEAN ANNE' '320 FORT DUQUESNE BLVD UNIT B&C, PITTSBURGH, PA 15222' = True True True
0001C01662024300: '2015-09-04' 'HATTLER JEAN ANNE' '320 FORT DUQUESNE BLVD UNIT B&C, PITTSBURGH, PA 15222' = True True True
0001C01662024400: '2006-07-13' 'PALUCKA CONNIE S & TIMOTHY P (H)' '320 FORT DUQUESNE BLVD UNIT 24-D, PITTSBURGH, PA 15222' = True True True
0001C01662024600: '2015-12-14' 'DAMASIO BRUCE L' '320 FORT DUQUESNE BLVD UNIT 24F, PITTSBURGH, PA 15222' = True True True
0001C01662024700: '2004-06-21' 'KASHKOUSH MOUNIR M' '320 FORT DUQUESNE BLVD UNIT 24-G, PITTSBURGH, PA 15222' = True True True
0001C01662024800: '2016-12-02' 'MASSARO JAMES' '320 FORT DUQUESNE BLVD UNIT 24-HJ, PITTSBURGH, PA 15222' = True True True
0001C01662004200: '2011-06-27' 'DR DIETER BARICH ERBENIMMOBILIENVERWALTUNG GMBH & C0 KG' 'PFORTZER ST 1 A 54439 SARBURG, GERMANY' = 

0011J00048000000: '1986-08-19' 'REILLY LAWRENCE W & ROSE E (W)' '1901 FORBES ST, PITTSBURGH, PA 15219' = True True True
0011J00049000000: '2005-08-19' 'KEENAN CHARLES J' '115 GIST ST, PITTSBURGH, PA 15219' = True True True
0011J00052000000: '2010-03-17' 'KEENAN CHARLES J' '115 GIST ST, PITTSBURGH, PA 15219' = True True True
0011J00053000000: '2007-08-06' 'KEENAN CHARLEY' '115 GIST ST, PITTSBURGH, PA 15219' = True True True
0011J00068000000: '2011-10-20' 'SHANAHAN HOUSING ASSOCIATES LP' '803 EAST PITTSBURGH MALL, EAST PITTSBURGH, PA 15112' = True True True
0011J00069000000: '1998-10-07' 'FENICATO SALVATORE & GIROLAMA (W)' '1708 WATSON ST, PITTSBURGH, PA 15219' = True True True
0011J00069000B00: '2010-11-22' 'BRGS LLC' 'PO BOX 13444, PITTSBURGH, PA 15243' = True True True
0011J00069000C00: '1995-08-03' 'KIRKAVITCH MARY & SANDRA M KIRKAVITCH' '1735 FORBES AVE, PITTSBURGH, PA 15219' = True True True
0011J00069000D00: '1984-08-30' 'GOLDSTON EDWARD M & JAMES L SPROAT' '1012 E CARSON ST, PITT

...

0011E00321000000: '2012-08-08' 'DINWIDDIE HOUSING LIMITEDPARTNERSHIP III' '130 7TH ST STE 300, PITTSBURGH, PA 15222' = True True True
0010N00109000001: '1978-05-23' 'SMITH PHILLIP & MADELYN T (W)' '174 GRANVILLE ST, PITTSBURGH, PA 15219' = True True True
0010N00110000000: '1985-12-27' 'CHAMBERS LESLIE A' '177 SWEENEY WAY, PITTSBURGH, PA 15219' = True True True
0010N00112000000: '1978-04-17' 'HEFFLIN CLYDE T & PATRICIA C' '162 GRANVILLE ST, PITTSBURGH, PA 15219' = True True True
0010N00116000000: '2002-12-30' 'HEFFLIN JALLA A' '161 SWEENEY ST, PITTSBURGH, PA 15219' = True True True
0011E00323000000: '2012-08-08' 'DINWIDDIE HOUSING LIMITEDPARTNERSHIP III' '130 7TH ST STE 300, PITTSBURGH, PA 15222' = True True True
0011E00325000000: '2012-08-08' 'DINWIDDIE HOUSING LIMITEDPARTNERSHIP III' '130 7TH ST STE 300, PITTSBURGH, PA 15222' = True True True
0011E00354000000: '2012-08-08' 'DINWIDDIE HOUSING LIMITEDPARTNERSHIP III' '130 7TH ST STE 300, PITTSBURGH, PA 15222' = True True True
0011E00355

0011E00227000000: '2012-07-16' 'SIMPSON SERENA THEA' '215 WICK ST, PITTSBURGH, PA 15219' = True True True
0011F00173000000: '2009-08-28' 'WILLIAMS TONY' '1 ELWYN AVE, CARNEGIE, PA 15106' = True True True
0011E00175000000: '2010-09-30' 'MARSHALL SHIRLEY ANN' '219 LOMBARD ST, PITTSBURGH, PA 15219' = True True True
0011E00176000000: '2016-03-02' 'BURTCH MARGARET' '215 LOMBARD ST, PITTSBURGH, PA 15219' = True True True
0011E00177000000: '2004-09-29' 'BURTCH MARGARET' '215 LOMBARD ST, PITTSBURGH, PA 15219' = True True True
0011A00168000000: '2014-07-15' 'ISLA BIRD 1 LLC' '5124 5TH AVE, PITTSBURGH, PA 15232' = True True True
0011A00170000000: '2014-02-14' 'URBAN REDEVELOPMENT AUTH OF PITTSBURGH' '200 ROSS ST FL 10, PITTSBURGH, PA 15219' = True True True
0011A00171000000: '2003-11-14' 'SWOOPE TAMIKA' 'PO BOX 53012, PITTSBURGH, PA 15219' = True True True
0011A00174000000: '2013-11-04' 'DINWIDDIE HOUSING LP IV' '130 7TH ST STE 300, PITTSBURGH, PA 15222' = True True True
0011A00175000000: '2015-

0011L00019000000: '2007-05-11' 'SCHOYER CLAIRE' '5439 ROSETTA ST, PITTSBURGH, PA 15206' = True True True
0011L00020000000: '1985-10-07' 'CITY OF PITTSBURGH' '414 GRANT ST RM 200, CITY-COUNTY BUILDING, PITTSBURGH, PA 15219' = True True True
0011L00025000000: '2007-05-11' 'CITY OF PITTSBURGH' '414 GRANT ST RM 200, CITY-COUNTY BUILDING, PITTSBURGH, PA 15219' = True True True
0011L00026000000: '1993-06-21' 'CITY OF PITTSBURGH' '414 GRANT ST RM 200, CITY-COUNTY BUILDING, PITTSBURGH, PA 15219' = True True True
0027D00008000000: '2012-07-27' 'FONTES JORGE E & MYRIAM (W)' '211 SAINT IVES WAY, ZELIENOPLE, PA 16063' = True True True
0027D00010000000: '2011-03-25' 'ROBERT V ERICKSON TRUST' '7449 BEN HUR ST, PITTSBURGH, PA 15208' = True True True
0011L00148000000: '2010-05-07' 'DUQUESNE LIGHT COMPANY' '1800 SEYMOUR ST, PITTSBURGH, PA 15233' = True True True
0027D00289000000: '1999-04-16' 'ECKENRODE ROBERT C & LINDA S (W)' '264 VEE LYNN DR, PITTSBURGH, PA 15228' = True True True
0027D00292000000: '

0011G00011000000: '1996-09-18' 'SCOTT RUBY V' '149 MOULTRIE ST, PITTSBURGH, PA 15219' = True True True
0011G00013000000: '2017-05-16' 'LEWIS LINDA ROBIN' '145 MOULTRIE ST, PITTSBURGH, PA 15219' = True True True
0011G00014000000: '2015-09-22' 'CITY OF PITTSBURGH' '414 GRANT ST RM 200, CITY-COUNTY BUILDING, PITTSBURGH, PA 15219' = True True True
0011G00015000000: '1984-08-17' 'SZABO JOSEPH A' '141 MOULTRIE ST, PITTSBURGH, PA 15219' = True True True
0011G00016000000: '1993-02-23' 'TYLER JOELANN M' '139 MOULTRIE ST, PITTSBURGH, PA 15219' = True True True
0011G00017000000: '2010-11-15' 'TYLER TAMEKA' '137 MOULTRIE ST, PITTSBURGH, PA 15219' = True True True
0011G00018000000: '2015-08-14' 'CITY OF PITTSBURGH' '414 GRANT ST RM 200, CITY-COUNTY BUILDING, PITTSBURGH, PA 15219' = True True True
0011G00019000000: '2015-08-14' 'CITY OF PITTSBURGH' '414 GRANT ST RM 200, CITY-COUNTY BUILDING, PITTSBURGH, PA 15219' = True True True
0011G00021000000: '1998-06-19' 'HAYS GARY D' '129 MOULTRIE ST, PITTSBU

ValueError: year out of range

In [453]:
apd_2009[apd_2009.PARID=='1421B00120000000'].PROPERTYOWNER

492959    MAKEPEACE JACK A & PATRICIA L (W)
Name: PROPERTYOWNER, dtype: object

In [215]:
apd_2017[apd_2017.index==3888]['SALEDATE']

3888    03-06-1890
Name: SALEDATE, dtype: object

In [ ]:
for i in range(3886,len(apd_2017)):
    print i
    process_assessment_record(apd_2017,i)

In [58]:
jw = apd_2017[apd_2017.PROPERTYOWNER=='WANG JANET F']

for i in range(0, len(jw.index.values)):
    idx=jw.index.values[i]
    process_assessment_record(jw,idx)

0028H00031000000: '2015-03-19' 'WANG JANET F' '3711   ORPWOOD ST, PITTSBURGH PA, 15213' = False False False
0029C00304000000: '2006-05-17' 'WANG JANET F' '3711   ORPWOOD ST, PITTSBURGH PA, 15213' = False False False
0029G00251000000: '1989-09-15' 'WANG JANET F' '3711   ORPWOOD ST, PITTSBURGH PA, 15213' = False False False
0029C00250000000: '1998-10-23' 'WANG JANET F' '3711   ORPWOOD ST, PITTSBURGH PA, 15213' = False False False
0029C00250000A00: '1998-10-23' 'WANG JANET F' '3711   ORPWOOD ST, PITTSBURGH PA, 15213' = False False False
0029H00119000000: '1996-07-24' 'WANG JANET F' '3711   ORPWOOD ST, PITTSBURGH PA, 15213' = False False False
0029H00121000000: '1990-05-30' 'WANG JANET F' '3711   ORPWOOD ST, PITTSBURGH PA, 15213' = False False False
0029C00285000000: '2002-11-06' 'WANG JANET F' '3711   ORPWOOD ST, PITTSBURGH PA, 15213' = False False False
0029H00037000000: '2016-04-29' 'WANG JANET F' '3711   ORPWOOD ST, PITTSBURGH PA, 15213' = False False False
0029H00038000000: '2016-04-2

# Deal with older versions of the assessment database from the ACCDB files from Bob Gradeck

In [63]:
# From https://stackoverflow.com/questions/17123550/extract-and-sort-data-from-mdb-file-using-mdbtools-in-python?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
import sys, subprocess, os
from io import StringIO
import pandas as pd
VERBOSE = True
def mdb_to_pandas(database_path):
    subprocess.call(["mdb-schema", database_path, "mysql"])
    # Get the list of table names with "mdb-tables"
    table_names = subprocess.Popen(["mdb-tables", "-1", database_path],
                                   stdout=subprocess.PIPE).communicate()[0]
    tables = table_names.splitlines()
    sys.stdout.flush()
    # Dump each table as a stringio using "mdb-export",
    out_tables = {}
    for rtable in tables:
        table = rtable.decode()
        if VERBOSE: print('running table:',table)
        if table != '':
            if VERBOSE: print("Dumping " + table)
            contents = subprocess.Popen(["mdb-export", database_path, table],
                                        stdout=subprocess.PIPE).communicate()[0]
            temp_io = StringIO(contents.decode())
            print(table, temp_io)
            out_tables[table] = pd.read_csv(temp_io)
    return out_tables
pd.options.display.max_colwidth = 300
pd.set_option('display.max_columns', 500)

In [64]:
# ACCDB files are from Bob Gradeck
# 2009 is missing YEARBLT, CLASSDESC, and CLASS
accdb_info = {'2009':{'fname':'assessments/gradeck/AssessmentSep09.mdb',
                      'tname':'pncis',
                     'col_remap': {'PIN':'PARID',
                                    'FAIRMARK_2':'FAIRMARKETTOTAL',
                                    'USEDESC':'USEDESC',
                                    'PROPERTYZI':'PROPERTYZIP',
                                    'PROPERTYOW':'PROPERTYOWNER',
                                    'CHANGENOTI':'CHANGENOTICEADDRESS1',
                                    'CHANGENO_1':'CHANGENOTICEADDRESS2',
                                    'CHANGENO_2':'CHANGENOTICEADDRESS3',
                                    'CHANGENO_3':'CHANGENOTICEADDRESS4',
                                    'HOMESTEADF':'HOMESTEADFLAG',
                                    'OWNERDESC':'OWNERDESC',
                                    'SALEDATE':'SALEDATE',
                                    'SALEPRICE':'SALEPRICE',
                                  }},
               '2010':'assessments/gradeck/June2010AssessNew.accdb',
               '2011':'assessments/gradeck/AssessMarch2011Data3.mdb.accdb',
               '2012':'assessments/gradeck/AssessNov2012.mdb',
               '2013':'assessments/gradeck/Dec2013Assess.accdb',
               '2014':'assessments/gradeck/June2014Assess.accdb',
               '2015':{'fname':'assessments/gradeck/May2015Assess.accdb',
                       'tname':'RawDataAssessment'
                       'col_remap': {'ChangeNoticeFullAddress1':'CHANGENOTICEADDRESS1',
                                    'ChangeNoticeFullAddress2':'CHANGENOTICEADDRESS2',
                                    'ChangeNoticeFullAddress3':'CHANGENOTICEADDRESS3',
                                    'ChangeNoticeFullAddress4':'CHANGENOTICEADDRESS4',
                                    
                                    },
                      }
             }

In [ ]:
accdb_2015_map = mdb_to_pandas(accdb_info['2015']['fname'])     
bpd_2015 = accdb_2015_map[accdb_info['2015']['tname']]
bpd_2015

In [ ]:
accdb_2009_map = mdb_to_pandas(accdb_info['2009']['fname'])     
bpd_2009 = accdb_2009_map[accdb_info['2009']['tname']]

In [66]:
apd_2009 = bpd_2009.rename(index=str, columns=accdb_info['2009']['col_remap'])
apd_2009

,PARID,MAPBLOCKLO,MUNICODE,PSEUDONO,PROPERTYOWNER,PROPERTYAD,PROPERTYHO,PROPERTY_1,PROPERTYUN,PROPERTYLO,PROPERTYCI,PROPERTYZIP,FAIRMARKET,FAIRMARK_1,FAIRMARKETTOTAL,COUNTYLAND,COUNTYBUIL,EXEMPTAMOU,PRIORMARKE,MUNDESC,SCHOOLCODE,SCHOOLDESC,TAXCODE,TAXDESC,OWNERCODE,OWNERDESC,STATECODE,STATEDESC,USECODE,USEDESC,CHANGENOTICEADDRESS1,CHANGENOTICEADDRESS2,CHANGENOTICEADDRESS3,CHANGENOTICEADDRESS4,AGENTNAME,SERVICECOD,TAXBILLFUL,TAXBILLF_1,TAXBILLF_2,TAXBILLF_3,SALEDATE,SALEPRICE,DEEDBOOK,DEEDPAGE,SALECODE,SALEDESC,HOMESTEADFLAG,FARMSTEADF,MULTIPLEAB,ACT77INDIC,LOCALEXEMP,LANDTYPE,LOTAREA,SHAPE_area,SHAPE_len,SHAPE_fid
0,0040A00310000000,40-A-310,835.0,NaN,HATT MILLICENT H & SANDRA GIBBS,NaN,417,UNION AVE,NaN,NaN,"PITTSBURGH, PA",15205.0,52800.0,16500.0,69300.0,16500.0,37800.0,0.0,69300.0,INGRAM,24.0,Montour,T,Taxable,10.0,Regular,R,Residential,10.0,SINGLE FAMILY,NaN,417 UNION AVE,"PITTSBURGH, PA 15205-2425",NaN,NaN,NaN,NaN,417 UNION AVE,"PITTSBURGH, PA 15205-",NaN,09/16/96 00:00:00,1.0,9783.0,45,3,LOVE AND AFFECTION SALE,C,NaN,NaN,Y,0.0,NaN,5822.0,5926.778809,358.015852,432259.0
1,0040A00352000000,40-A-352,835.0,NaN,BAKER JEFFREY L & JUDITH (W),NaN,0,CENTER ST,NaN,NaN,"PITTSBURGH, PA",15205.0,0.0,18700.0,18700.0,18700.0,0.0,0.0,18700.0,INGRAM,24.0,Montour,T,Taxable,12.0,Regular,R,Residential,100.0,VACANT LAND,NaN,294 CRAFTON BLVD,"PGH, PA 15205-2571",NaN,NaN,NaN,NaN,294 CRAFTON BLVD,"PITTSBURGH, PA 15205-0000",NaN,08/04/88 00:00:00,40000.0,7842.0,185,0,VALID SALE,NaN,NaN,NaN,NaN,0.0,NaN,9900.0,9905.212330,400.114341,468692.0
2,0040B00110000000,40-B-110,128.0,NaN,DISANTI WILLIAM J JR & MARY JO (W),NaN,1326,HOLLYWOOD ST,NaN,NaN,"PITTSBURGH, PA",15205.0,52800.0,4200.0,57000.0,4200.0,37800.0,0.0,57000.0,PITTSBURGH - 28TH WARD,47.0,City Of Pittsburgh,T,Taxable,12.0,Regular,R,Residential,70.0,ROWHOUSE,NaN,1326 HOLLYWOOD ST,"PITTSBURGH, PA 15205-3427",NaN,DISANTI WILLIAM J JR & MARY JO (W),NaN,NaN,1326 HOLLYWOOD ST,"PITTSBURGH, PA 15205-",NaN,04/13/88 00:00:00,63000.0,7761.0,432,9,OTHER INVALID SALES INDICATED,C,NaN,NaN,NaN,0.0,NaN,5000.0,4831.146352,298.285632,1015170.0
3,1671S00064000000,1671-S-64,952.0,NaN,SIWICKI ROSE B,NaN,0,DEER CREEK RD,NaN,NaN,"GIBSONIA, PA",15044.0,0.0,21400.0,21400.0,21400.0,0.0,0.0,21400.0,WEST DEER,12.0,Deer Lakes,T,Taxable,10.0,Regular,R,Residential,100.0,VACANT LAND,NaN,302 FIFTH AVE,"CARNEGIE, PA 15106-2325",NaN,NADINE SIWICKI,NaN,1ST FL,302 5TH AVE,"CARNEGIE, PA 15106-0000",NaN,12/13/93 00:00:00,0.0,9116.0,98,3,LOVE AND AFFECTION SALE,NaN,NaN,NaN,NaN,0.0,NaN,194408.0,205512.387874,2413.694528,1442927.0
4,0040B00041000000,40-B-41,128.0,NaN,CAREY KEVIN J & LISA M CAREY (W),NaN,847,FORD ST,NaN,NaN,"PITTSBURGH, PA",15205.0,53200.0,6800.0,60000.0,6800.0,38200.0,0.0,60000.0,PITTSBURGH - 28TH WARD,47.0,City Of Pittsburgh,T,Taxable,12.0,Regular,R,Residential,10.0,SINGLE FAMILY,NaN,847 FORD ST,"PGH, PA 15205-3519",NaN,NaN,NaN,NaN,847 FORD ST,"PITTSBURGH, PA 15205-0000",NaN,07/08/98 00:00:00,67000.0,10242.0,155,H,MULTI-PARCEL SALE,C,NaN,NaN,NaN,0.0,NaN,3000.0,3129.883596,291.239986,1015108.0
5,0040B00119000000,40-B-119,128.0,NaN,SEBESTA JOHN C & VIRGINIA ANNA,NaN,1729,STRATMORE AVE,NaN,NaN,"PITTSBURGH, PA",15205.0,58100.0,7100.0,65200.0,7100.0,43100.0,0.0,65200.0,PITTSBURGH - 28TH WARD,47.0,City Of Pittsburgh,T,Taxable,10.0,Regular,R,Residential,10.0,SINGLE FAMILY,NaN,1729 STRATMORE AVE,"PGH, PA 15205-3454",NaN,NaN,NaN,NaN,1729 STRATMORE AVE,"PITTSBURGH, PA 15205-0000",NaN,05/05/50 00:00:00,0.0,0.0,0,NaN,NaN,C,NaN,NaN,NaN,0.0,NaN,2500.0,2402.112386,246.223111,1007645.0
6,0542C00094000000,542-C-94,953.0,NaN,DEMARCO ANTHONY J & CARLA A DEMARCO,NaN,200,HART DR,NaN,NaN,"PITTSBURGH, PA",15235.0,54200.0,18500.0,72700.0,18500.0,39200.0,0.0,72700.0,WILKINS,9.0,Woodland Hills,T,Taxable,10.0,Regular,R,Residential,10.0,SINGLE FAMILY,NaN,200 HART DR,"PITTSBURGH, PA 15235-5426",NaN,NaN,NaN,NaN,200 HART DR,"PITTSBURGH, PA 15235-0000",NaN,08/29/96 00:00:00,1.0,9770.0,401,3,LOVE AND AFFECTION SALE,C,NaN,NaN,NaN,0.0,NaN,7123.0,7202

In [239]:
start=arrow.now()
for parid in res_parids:
    index = 
    print parid
    

In [251]:
apd_2009.index[apd_2009.PARID==res_parids[1]].tolist()

[]

In [240]:
end=arrow.now()

In [244]:
delta=end-start
str(delta)

'0:00:31.496021'

In [253]:
# Add entries from apd_2009
start=arrow.now()
for i in range(0,len(apd_2009)):
    process_assessment_record(apd_2009,i)
end=arrow.now()
print "Processing took %s" % (str(end-start))

Processing took 0:56:55.373069


In [174]:
len(apd_2009)

570106

In [176]:
len(apd_2017)

578149

In [254]:
len(property_map)

514145

In [255]:
len(res_parids)

519728

In [256]:
len(owner_map)

586452

In [257]:
len(changeaddr_map)

535838

In [454]:
property_map['0080K00240000A00']

[{'changeaddr': '32 CAMBRIA POINT ST, PITTSBURGH, PA 15209',
  'date': u'1996-11-19',
  'event_type': 'PURCHASE',
  'ownername': 'PUPICH CHARLES'},
 {'changeaddr': '32 CAMBRIA PT, PITTSBURGH, PA 15209',
  'date': u'1996-11-19',
  'event_type': 'PURCHASE',
  'ownername': 'PUPICH CHARLES'}]

In [456]:
owner_map['LAWRENCEVILLE CORPORATION']

[{'date': u'2005-10-20',
  'event_type': 'PURCHASE',
  'parid': '0049N00147000000'},
 {'date': u'2008-04-11',
  'event_type': 'PURCHASE',
  'parid': '0119S00063000000'},
 {'date': u'2008-05-23',
  'event_type': 'PURCHASE',
  'parid': '0119S00053000000'},
 {'date': u'2008-07-14',
  'event_type': 'PURCHASE',
  'parid': '0080G00287000000'},
 {'date': u'2009-06-17',
  'event_type': 'PURCHASE',
  'parid': '0049N00144000000'},
 {'date': u'2010-03-26', 'event_type': 'SALE', 'parid': '0049N00147000000'},
 {'date': u'2012-02-01', 'event_type': 'SALE', 'parid': '0119S00063000000'},
 {'date': u'2012-02-01', 'event_type': 'SALE', 'parid': '0119S00053000000'},
 {'date': u'2013-09-05', 'event_type': 'SALE', 'parid': '0049N00144000000'},
 {'date': u'2015-02-25',
  'event_type': 'PURCHASE',
  'parid': '0080H00151000000'},
 {'date': u'2015-05-15',
  'event_type': 'PURCHASE',
  'parid': '0049E00020000000'},
 {'date': u'2015-05-15',
  'event_type': 'PURCHASE',
  'parid': '0048S00192000000'},
 {'date': u'

In [375]:
# For a common parcel id, check if two events are both for the same transaction.  
# This happens when the dates are the same and/or when the set of owner names overlap
def is_same_transaction(e1,e2):
    if(e1['event_type']!='PURCHASE' or e2['event_type']!='PURCHASE'):
        return False
    if(e1['date']==e2['date']):
        return True
    if((e1['ownername'] in e2['ownername']) or (e2['ownername'] in e1['ownername'])):
        return True
    return False
    
def register_sale_dates(parid):
    prop_ev_list = property_map[parid]
    if(len(prop_ev_list)<2):
        # No sales have happened that we know about, only purchases
        return

    # Keep a list of the equivalent transactions so any subsequent sale can 
    # be applied to all.  Seed it with the 0'th item in prop_ev_list
    equiv_transactions=[prop_ev_list[0]]
    #print "Starting with %r" %(equiv_transactions)
    for i in range(1,len(prop_ev_list)):
        # See if this new transaction is equivalent to the last item in the current list
        if(is_same_transaction(prop_ev_list[i],equiv_transactions[-1])):
            equiv_transactions.append(prop_ev_list[i])
            #print "  %d still equivalent %r" %(i, equiv_transactions)
        else:
            #print "  %d not equivalent %r, record sale date" %(i,prop_ev_list[i])
            sell_event={'date':prop_ev_list[i]['date'],'event_type':'SALE','parid':parid}
            
            for j in range(0,len(equiv_transactions)):
                    dup_o = insert_event(owner_map[equiv_transactions[j]['ownername']], sell_event)
                    dup_c = insert_event(changeaddr_map[equiv_transactions[j]['changeaddr']], sell_event)
                    #print "    %d %r: %r %r" % (j, equiv_transactions[j], dup_o, dup_c)
            # Reset equiv_transactions and keep going if any more transactions are left
            equiv_transactions=[prop_ev_list[i]]
        
# This needs to be called once after imports are complete and before get_owned_parids
def register_all_sales():
    for parid in property_map.keys():
        register_sale_dates(parid)
        
def get_owned_parids(event_list, eval_date):
    # Use a set for accumulating parids since we don't want duplicates
    owned_parids=set()
    # In case eval_date is at or after the last date in the list, 
    # default next_date to today
    next_date = this_date
    for i in range(0,len(event_list)):
        if(event_list[i]['date']<= eval_date):
            # This event happened on or before the date we're asking about, process the event
            if(event_list[i]['event_type']=='PURCHASE'):
                # Add purchased property to owned_parids
                owned_parids.add(event_list[i]['parid'])
            else:
                # This must be a sale, remove it from owned_parids
                owned_parids.remove(event_list[i]['parid'])
        else:
            # This and subsequent events happened after the date we're looking for.
            # Set next_date and return owned_parids
            next_date = event_list[i]['date']
            break

    return {'parids':owned_parids,'next_date':next_date}

# For a given parid and eval_date, get a map back containing:
#   'parids' = the set of parids owned by the same owners/changeaddrs as of eval_date, and
#   'next_date' = the next date that something changes within that set of owners/changeaddrs
# When the last of the related events is complete, 'next_date'==this_date
def get_related_parids(parid, eval_date):
    prop_ev_list = property_map[parid]
    parid_set = set()
    start_date = eval_date
    next_date = this_date

    # Keep a list of the equivalent transactions so we can accumulate the 
    # parids for all.  Seed it with the 0'th item in prop_ev_list
    equiv_transactions=[prop_ev_list[0]]
    # If date of the first event is later than eval_date, set eval_date to 
    # the first event.  We don't want to iterate over the earlier purchases by
    # the original owners we know about.  Just start at the first purchase
    # of this parcel we know about.
    if(eval_date<prop_ev_list[0]['date']):
        eval_date=prop_ev_list[0]['date']
        start_date=eval_date
    #print "%s: starting with %r" %(parid, equiv_transactions)
    for i in range(1,len(prop_ev_list)):
        # See if this new transaction is equivalent to the last item in the current list
        if(is_same_transaction(prop_ev_list[i],equiv_transactions[-1]) and 
           prop_ev_list[i]['date']<=eval_date):
            equiv_transactions.append(prop_ev_list[i])
            #print "  %d still equivalent %r" %(i, equiv_transactions)
        elif(prop_ev_list[i]['date']>eval_date):
            #print "  %d after timespan (%r), process current set" %(i,prop_ev_list[i])
            break
        else:
            # We haven't hit the end of the timespan yet, but the ownership has changed.
            # Reset equiv_transactions starting from the current event
            equiv_transactions=[prop_ev_list[i]]
            
    # We've got all the equivalent transactions, process them
    changeaddr_set=set()
    owner_set=set()
    for j in range(0,len(equiv_transactions)):
        owner_set.add(equiv_transactions[j]['ownername'])
        changeaddr_set.add(equiv_transactions[j]['changeaddr'])

    #print "    Owner set= %r\n    Changeaddr set= %r" % (owner_set, changeaddr_set)
    
    for owner in owner_set:
        ret_map=get_owned_parids(owner_map[owner],eval_date)
        parid_set = parid_set.union(ret_map['parids'])
        if(ret_map['next_date']<next_date):
            next_date = ret_map['next_date']
    for changeaddr in changeaddr_set:
        ret_map=get_owned_parids(changeaddr_map[changeaddr],eval_date)
        parid_set = parid_set.union(ret_map['parids'])
        if(ret_map['next_date']<next_date):
            next_date = ret_map['next_date']

    ret_val={"parids":parid_set, "parcount":len(parid_set), "start_date":start_date, "next_date":next_date}
    #print "    Returning %r" % (ret_val)
    return ret_val

In [376]:
start=arrow.now()
register_all_sales()
end=arrow.now()
print "Processing took %s" % (str(end-start))

Processing took 0:00:04.532053


In [379]:
next_date = '1970-01-01'
parid='0298G00064000000'
while(next_date<this_date):
    ret=get_related_parids(parid, next_date)
    print "%s: %s - %s = %d" % (parid, ret['start_date'], ret['next_date'], ret['parcount'])
    next_date=ret['next_date']

0298G00064000000: 1999-07-01 - 2002-12-09 = 4
0298G00064000000: 2002-12-09 - 2016-04-29 = 5
0298G00064000000: 2016-04-29 - 2018-08-08 = 3


In [368]:
changeaddr_map['3817 BATES ST, PGH, PA 15213']=[{'date': u'1985-11-08',
  'event_type': 'PURCHASE',
  'parid': '0028H00031000000'},
 {'date': u'2015-03-19', 'event_type': 'SALE', 'parid': '0028H00031000000'}]

In [371]:
changeaddr_map['2431 GREENSBURG PIKE, PITTSBURGH, PA 15221']=[{'date': u'1973-10-12',
  'event_type': 'PURCHASE',
  'parid': '0057C00343000000'},
 {'date': u'1991-06-27',
  'event_type': 'PURCHASE',
  'parid': '0230H00190000000'},
 {'date': u'1994-08-31',
  'event_type': 'PURCHASE',
  'parid': '0029H00038000000'},
 {'date': u'1999-07-01',
  'event_type': 'PURCHASE',
  'parid': '0298G00064000000'},
 {'date': u'2002-12-09',
  'event_type': 'PURCHASE',
  'parid': '0029H00037000000'},
 {'date': u'2016-04-29', 'event_type': 'SALE', 'parid': '0029H00037000000'},
 {'date': u'2016-04-29', 'event_type': 'SALE', 'parid': '0029H00038000000'}]

In [353]:
changeaddr_map['16 BAKER RD, LAKE LYNN, PA 15451']=[{'date': u'1984-07-06',
  'event_type': 'PURCHASE',
  'parid': '0051S00006000000'},
 {'date': u'1989-09-15',
  'event_type': 'PURCHASE',
  'parid': '0029G00251000000'},
 {'date': u'1989-10-10',
  'event_type': 'PURCHASE',
  'parid': '0028S00106000000'},
 {'date': u'1990-05-30',
  'event_type': 'PURCHASE',
  'parid': '0029H00121000000'},
 {'date': u'1991-05-17',
  'event_type': 'PURCHASE',
  'parid': '0029G00285000000'},
 {'date': u'1991-10-16',
  'event_type': 'PURCHASE',
  'parid': '0029S00063000000'},
 {'date': u'1996-07-24',
  'event_type': 'PURCHASE',
  'parid': '0029H00119000000'},
 {'date': u'1997-11-24',
  'event_type': 'PURCHASE',
  'parid': '0029D00076000000'},
 {'date': u'1998-04-16',
  'event_type': 'PURCHASE',
  'parid': '0029G00268000000'},
 {'date': u'1998-10-23',
  'event_type': 'PURCHASE',
  'parid': '0029C00250000A00'},
 {'date': u'1998-10-23',
  'event_type': 'PURCHASE',
  'parid': '0029C00250000000'},
 {'date': u'2000-01-25',
  'event_type': 'PURCHASE',
  'parid': '0029G00269000000'},
 {'date': u'2000-02-22',
  'event_type': 'PURCHASE',
  'parid': '0029G00274000000'},
 {'date': u'2001-03-20',
  'event_type': 'PURCHASE',
  'parid': '0028S00105000000'},
 {'date': u'2001-09-28',
  'event_type': 'PURCHASE',
  'parid': '0029H00120000000'},
 {'date': u'2002-11-06',
  'event_type': 'PURCHASE',
  'parid': '0029C00285000000'},
 {'date': u'2002-12-10',
  'event_type': 'PURCHASE',
  'parid': '0029G00284000000'},
 {'date': u'2006-05-17',
  'event_type': 'PURCHASE',
  'parid': '0029C00304000000'},
 {'date': u'2006-06-02',
  'event_type': 'PURCHASE',
  'parid': '0029C00290000000'},
 {'date': u'2006-06-02',
  'event_type': 'PURCHASE',
  'parid': '0029C00301000000'},
 {'date': u'2006-12-22',
  'event_type': 'PURCHASE',
  'parid': '0029G00283000000'},
 {'date': u'2009-06-24',
  'event_type': 'PURCHASE',
  'parid': '0029C00286000000'},
 {'date': u'2015-05-08',
  'event_type': 'PURCHASE',
  'parid': '0029H00040000000'},
 {'date': u'2016-07-20', 'event_type': 'SALE', 'parid': '0029G00269000000'}]

In [372]:
changeaddr_map['3711 ORPWOOD ST, PITTSBURGH, PA 15213']

[{'date': u'1984-07-06',
  'event_type': 'PURCHASE',
  'parid': '0051S00006000000'},
 {'date': u'1989-09-15',
  'event_type': 'PURCHASE',
  'parid': '0029G00251000000'},
 {'date': u'1989-10-10',
  'event_type': 'PURCHASE',
  'parid': '0028S00106000000'},
 {'date': u'1990-05-30',
  'event_type': 'PURCHASE',
  'parid': '0029H00121000000'},
 {'date': u'1991-05-17',
  'event_type': 'PURCHASE',
  'parid': '0029G00285000000'},
 {'date': u'1996-07-24',
  'event_type': 'PURCHASE',
  'parid': '0029H00119000000'},
 {'date': u'1997-11-24',
  'event_type': 'PURCHASE',
  'parid': '0029D00076000000'},
 {'date': u'1998-04-16',
  'event_type': 'PURCHASE',
  'parid': '0029G00268000000'},
 {'date': u'1998-10-23',
  'event_type': 'PURCHASE',
  'parid': '0029C00250000000'},
 {'date': u'1998-10-23',
  'event_type': 'PURCHASE',
  'parid': '0029C00250000A00'},
 {'date': u'2000-02-22',
  'event_type': 'PURCHASE',
  'parid': '0029G00274000000'},
 {'date': u'2001-09-28',
  'event_type': 'PURCHASE',
  'parid': '

In [442]:
get_owned_parids(owner_map['EDENS PROPERTY LLC'],'1980-01-01')

{'next_date': u'2009-06-24', 'parids': set()}

In [356]:
change_map['EDENS PROPERTY LLC']

[{'date': u'2009-06-24',
  'event_type': 'PURCHASE',
  'parid': '0029C00286000000'}]

In [320]:
owner_map['WANG STEPHEN P']=[{'date': u'2000-01-25',
  'event_type': 'PURCHASE',
  'parid': '0029G00269000000'},
 {'date': u'2016-07-20', 'event_type': 'SALE', 'parid': '0029G00269000000'}]

In [324]:
owner_map['RUE ROY']=[{'date': u'1991-06-27',
  'event_type': 'PURCHASE',
  'parid': '0230H00190000000'},
 {'date': u'1994-08-31',
  'event_type': 'PURCHASE',
  'parid': '0029H00038000000'},
 {'date': u'1999-07-01',
  'event_type': 'PURCHASE',
  'parid': '0298G00064000000'},
 {'date': u'2002-12-09',
  'event_type': 'PURCHASE',
  'parid': '0029H00037000000'},
 {'date': u'2016-04-29', 'event_type': 'SALE', 'parid': '0029H00037000000'},
 {'date': u'2016-04-29', 'event_type': 'SALE', 'parid': '0029H00038000000'}]

In [330]:
owner_map['ROSSI EUGENE D & NORMA JEAN (W)']=[{'date': u'1985-11-08',
  'event_type': 'PURCHASE',
  'parid': '0028H00031000000'},
 {'date': u'2015-03-19', 'event_type': 'SALE', 'parid': '0028H00031000000'}]

In [ ]:
for event in owner_map['WANG JANET F']:
    register_sale_dates(event['parid'])

In [331]:
get_owned_parids(owner_map['ROSSI EUGENE D & NORMA JEAN (W)'], '1986-04-29')

{'0028H00031000000'}

In [300]:
is_same_transaction({'date': u'2008-04-24', 'changeaddr': '7316 NOBLESTOWN RD, OAKDALE, PA 15071', 'event_type': 'PURCHASE', 'ownername': 'HOOKEY JAMEdS D'}, 
                   {'date': u'2008-04-24', 'changeaddr': '7316 NOBLESTOWN RD, OAKDALE, PA 15071', 'event_type': 'PURCHASE', 'ownername': 'HOOKEY JAMES D & CAROLYN M DLUGOSZ'})

True

In [308]:
register_sale_dates('0299M00109000000')

Starting with [{'date': u'2006-03-15', 'changeaddr': '120 PARISE RD, PITTSBURGH, PA 15221', 'event_type': 'PURCHASE', 'ownername': 'MITSCH MATTHEW & KELLY MITSCH'}]
  1 not equivalent {'date': u'2014-07-22', 'changeaddr': '120 PARISE RD, PITTSBURGH, PA 15221', 'event_type': 'PURCHASE', 'ownername': 'KOCH RICHARD E'}, record sale date
    0 {'date': u'2006-03-15', 'changeaddr': '120 PARISE RD, PITTSBURGH, PA 15221', 'event_type': 'PURCHASE', 'ownername': 'MITSCH MATTHEW & KELLY MITSCH'}: False False


In [309]:
owner_map['MITSCH MATTHEW & KELLY MITSCH']

[{'date': u'2006-03-15',
  'event_type': 'PURCHASE',
  'parid': '0299M00109000000'},
 {'date': u'2014-07-22', 'parid': '0299M00109000000', 'type': 'SALE'}]

In [277]:
apd_2017[apd_2017.PARID=='0232K00077000000']

,PARID,PROPERTYOWNER,PROPERTYHOUSENUM,PROPERTYFRACTION,PROPERTYADDRESS,PROPERTYCITY,PROPERTYSTATE,PROPERTYUNIT,PROPERTYZIP,MUNICODE,MUNIDESC,SCHOOLCODE,SCHOOLDESC,LEGAL1,LEGAL2,LEGAL3,NEIGHCODE,NEIGHDESC,TAXCODE,TAXDESC,TAXSUBCODE,TAXSUBCODE_DESC,OWNERCODE,OWNERDESC,CLASS,CLASSDESC,USECODE,USEDESC,LOTAREA,HOMESTEADFLAG,FARMSTEADFLAG,CLEANGREEN,ABATEMENTFLAG,RECORDDATE,SALEDATE,SALEPRICE,SALECODE,SALEDESC,DEEDBOOK,DEEDPAGE,PREVSALEDATE,PREVSALEPRICE,PREVSALEDATE2,PREVSALEPRICE2,AGENT,TAXFULLADDRESS1,TAXFULLADDRESS2,TAXFULLADDRESS3,TAXFULLADDRESS4,CHANGENOTICEADDRESS1,CHANGENOTICEADDRESS2,CHANGENOTICEADDRESS3,CHANGENOTICEADDRESS4,COUNTYBUILDING,COUNTYLAND,COUNTYTOTAL,COUNTYEXEMPTBLDG,LOCALBUILDING,LOCALLAND,LOCALTOTAL,FAIRMARKETBUILDING,FAIRMARKETLAND,FAIRMARKETTOTAL,STYLE,STYLEDESC,STORIES,YEARBLT,EXTERIORFINISH,EXTFINISH_DESC,ROOF,ROOFDESC,BASEMENT,BASEMENTDESC,GRADE,GRADEDESC,CONDITION,CONDITIONDESC,CDU,CDUDESC,TOTALROOMS,BEDROOMS,FULLBATHS,HALFBATHS,HEATINGCOOLING,HEATINGCOOLINGDESC,FIREPLACES,BSMTGARAGE,FINISHEDLIVINGAREA,CARDNUMBER,ALT_ID,TAXYEAR,ASOFDATE
222845,0232K00077000000,DAVIS ELMER JR,1453,,CRESSON ST,PITTSBURGH,PA,,15221,866,Wilkinsburg,46,Wilkinsburg Boro,WILKS ADDN PLAN PT 19 LOT 25X125 CRESSON ST,2 STY BRK & SHG HSE 1453,NaN,86607,WILKINSBURG BOROUGH,T,20 - Taxable,NaN,NaN,10,REGULAR,R,RESIDENTIAL,10,SINGLE FAMILY,3125,NaN,NaN,NaN,NaN,08-10-2006,08-10-2006,1.0,3,LOVE&AFFECTION,12951,143,05-05-1950,0.0,NaN,NaN,WELLS FARGO R E TAX SERVICES,1 HOME CAMPUS,MAC X 2302-04D,DES MOINES IA,50328,PO BOX 298,,ANNAPOLIS JUNCTION MD,20701,18800,3300,22100,0,18800,3300,22100,18800,3300,22100,16,OLD STYLE,2.0,1920.0,2.0,Brick,1.0,SHINGLE,5.0,Full,C-,AVERAGE -,4.0,FAIR,FR,FAIR,5.0,2.0,1.0,0.0,2,Central Heat,0.0,0.0,1200.0,1.0,NaN,2017,01-OCT-17


In [279]:
apd_2009[apd_2009.PARID=='0232K00077000000']

,PARID,MAPBLOCKLO,MUNICODE,PSEUDONO,PROPERTYOWNER,PROPERTYAD,PROPERTYHO,PROPERTY_1,PROPERTYUN,PROPERTYLO,PROPERTYCI,PROPERTYZIP,FAIRMARKET,FAIRMARK_1,FAIRMARKETTOTAL,COUNTYLAND,COUNTYBUIL,EXEMPTAMOU,PRIORMARKE,MUNDESC,SCHOOLCODE,SCHOOLDESC,TAXCODE,TAXDESC,OWNERCODE,OWNERDESC,STATECODE,STATEDESC,USECODE,USEDESC,CHANGENOTICEADDRESS1,CHANGENOTICEADDRESS2,CHANGENOTICEADDRESS3,CHANGENOTICEADDRESS4,AGENTNAME,SERVICECOD,TAXBILLFUL,TAXBILLF_1,TAXBILLF_2,TAXBILLF_3,SALEDATE,SALEPRICE,DEEDBOOK,DEEDPAGE,SALECODE,SALEDESC,HOMESTEADFLAG,FARMSTEADF,MULTIPLEAB,ACT77INDIC,LOCALEXEMP,LANDTYPE,LOTAREA,SHAPE_area,SHAPE_len,SHAPE_fid
437291,0232K00077000000,232-K-77,866.0,NaN,DAVIS ELMER JR,NaN,1453,CRESSON ST,NaN,NaN,"PITTSBURGH, PA",15221.0,19300.0,7100.0,26400.0,7100.0,4300.0,0.0,26400.0,WILKINSBURG,46.0,Wilkinsburg Boro,T,Taxable,10.0,Regular,R,Residential,10.0,SINGLE FAMILY,NaN,1453 CRESSON ST,"PGH, PA 15221-1927",NaN,DAVIS ELMER JR,NaN,NaN,1453 CRESSON ST,"PGH, PA 15221-0000",NaN,08/10/06 00:00:00,1.0,12951.0,143,3,LOVE AND AFFECTION SALE,C,NaN,NaN,NaN,0.0,NaN,3125.0,3225.535731,299.626155,1736616.0


In [280]:
get_owner_changeaddr(apd_2017,222845)

'PO BOX 298, ANNAPOLIS JUNCTION, MD 20701'

In [281]:
get_owner_changeaddr(apd_2009,437291)

'1453 CRESSON ST, PITTSBURGH, PA 15221'

In [187]:
process_assessment_record(apd_2009,725)

0385E00121000000: '2003-09-15' 'KENNEDY JOHN R & KATHLEEN MICHAELS KENNEDY' '823 BLACKBERRY ST, WEST MIFFLIN, PA 15122' = True True False


True

In [188]:
process_assessment_record(apd_2017,236404)

0385E00121000000: '2011-12-01' 'HEINZ LINDA L' '823 BLACKBERRY ST, WEST MIFFLIN, PA 15122' = False False False


False

In [189]:
owner_map['KENNEDY JOHN R                           <BR>KATHLEEN MICHAELS KENNEDY']

KeyError: 'KENNEDY JOHN R                           <BR>KATHLEEN MICHAELS KENNEDY'

In [190]:
property_map['0385E00121000000']

[{'changeaddr': '823 BLACKBERRY ST, WEST MIFFLIN, PA 15122',
  'date': '2003-09-15',
  'event_type': 'PURCHASE',
  'ownername': 'KENNEDY JOHN R KATHLEEN MICHAELS KENNEDY'},
 {'changeaddr': '823 BLACKBERRY ST, WEST MIFFLIN, PA 15122',
  'date': '2003-09-15',
  'event_type': 'PURCHASE',
  'ownername': 'KENNEDY JOHN R & KATHLEEN MICHAELS KENNEDY'},
 {'changeaddr': '823 BLACKBERRY ST, WEST MIFFLIN, PA 15122',
  'date': '2011-12-01',
  'event_type': 'PURCHASE',
  'ownername': 'HEINZ LINDA L'}]

In [191]:
property_map['0385E00118000000']

[{'changeaddr': '823 BLACKBERRY ST, WEST MIFFLIN, PA 15122',
  'date': '2003-09-15',
  'event_type': 'PURCHASE',
  'ownername': 'KENNEDY JOHN R KATHLEEN MICHAELS KENNEDY'}]

In [193]:
property_map['0385E00124000000']

[{'changeaddr': '823 BLACKBERRY ST, WEST MIFFLIN, PA 15122',
  'date': '2003-09-15',
  'event_type': 'PURCHASE',
  'ownername': 'KENNEDY JOHN R KATHLEEN MICHAELS KENNEDY'}]

In [423]:
property_map['0026F00356000000']

[{'changeaddr': 'CITY-COUNTY BUILDING, 414 GRANT ST RM 215, PITTSBURGH, PA 15219',
  'date': u'1950-05-05',
  'event_type': 'PURCHASE',
  'ownername': 'CITY OF PITTSBURGH'},
 {'changeaddr': '3607 COLMAR ST, PITTSBURGH, PA 15213',
  'date': u'2012-02-22',
  'event_type': 'PURCHASE',
  'ownername': 'SIBBET JOHN'}]

In [194]:
get_owner_changeaddr(apd_2009,400)

'160 REA LN, TARENTUM, PA 15084'

In [195]:
get_owner_changeaddr(apd_2017,491067)

'160 REA LN, TARENTUM, PA 15084'

In [196]:
apd_2017['CHANGENOTICEADDRESS2'][491067]

'  '

In [441]:
apd_2009[apd_2009.PARID=='0040B00132000000']['CHANGENOTICEADDRESS3'][0]

nan

In [440]:
bad_changeaddr=apd_2009.CHANGENOTICEADDRESS3==',  -'
apd_2009.loc[bad_changeaddr,'CHANGENOTICEADDRESS3']= numpy.nan

In [446]:
apd_2009[pandas.isna(apd_2009.CHANGENOTICEADDRESS1) & pandas.isna(apd_2009.CHANGENOTICEADDRESS2) & pandas.isna(apd_2009.CHANGENOTICEADDRESS3)& pandas.isna(apd_2009.CHANGENOTICEADDRESS4)]

,PARID,MAPBLOCKLO,MUNICODE,PSEUDONO,PROPERTYOWNER,PROPERTYAD,PROPERTYHO,PROPERTY_1,PROPERTYUN,PROPERTYLO,PROPERTYCI,PROPERTYZIP,FAIRMARKET,FAIRMARK_1,FAIRMARKETTOTAL,COUNTYLAND,COUNTYBUIL,EXEMPTAMOU,PRIORMARKE,MUNDESC,SCHOOLCODE,SCHOOLDESC,TAXCODE,TAXDESC,OWNERCODE,OWNERDESC,STATECODE,STATEDESC,USECODE,USEDESC,CHANGENOTICEADDRESS1,CHANGENOTICEADDRESS2,CHANGENOTICEADDRESS3,CHANGENOTICEADDRESS4,AGENTNAME,SERVICECOD,TAXBILLFUL,TAXBILLF_1,TAXBILLF_2,TAXBILLF_3,SALEDATE,SALEPRICE,DEEDBOOK,DEEDPAGE,SALECODE,SALEDESC,HOMESTEADFLAG,FARMSTEADF,MULTIPLEAB,ACT77INDIC,LOCALEXEMP,LANDTYPE,LOTAREA,SHAPE_area,SHAPE_len,SHAPE_fid
295,0162P00020000000,162-P-20,126.0,NaN,MADDOX WALTER III & FRANCES,NaN,0,ROOSEVELT AVE,NaN,NaN,"PITTSBURGH, PA",15214.0,0.0,1200.0,1200.0,1200.0,0.0,0.0,1200.0,PITTSBURGH - 26TH WARD,47.0,City Of Pittsburgh,T,Taxable,10.0,Regular,R,Residential,100.0,VACANT LAND,NaN,NaN,NaN,NaN,RF,NaN,NaN,MERRIMAN RD,"SEWICKLEY, PA 15143-0000",NaN,05/05/50 00:00:00,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2500.0,2594.354452,249.390715,1103632.0
325,0040B00132000000,40-B-132,128.0,NaN,PORTER ANNIE,NaN,0,STEUBENVILLE PIKE,NaN,NaN,"PITTSBURGH, PA",15205.0,0.0,1100.0,1100.0,1100.0,0.0,0.0,1100.0,PITTSBURGH - 28TH WARD,47.0,City Of Pittsburgh,T,Taxable,10.0,Regular,R,Residential,100.0,VACANT LAND,NaN,NaN,NaN,NaN,RF,NaN,NaN,11031 W MOUNTAIN VIEW RD,"SUN CITY, AZ 85351-0000",NaN,05/05/50 00:00:00,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2750.0,2997.422365,278.010181,1012938.0
340,0040B00100000000,40-B-100,128.0,NaN,SESSI HELEN,NaN,0,HOLLYWOOD ST,NaN,NaN,"PITTSBURGH, PA",15205.0,0.0,1000.0,1000.0,1000.0,0.0,0.0,1000.0,PITTSBURGH - 28TH WARD,47.0,City Of Pittsburgh,T,Taxable,10.0,Regular,R,Residential,100.0,VACANT LAND,NaN,NaN,NaN,NaN,RF,NaN,NaN,44 CHARTIERS ST,"RANKIN, PA 15104-0000",NaN,05/05/50 00:00:00,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2500.0,2409.965028,252.070297,1015094.0
439,0162P00035000000,162-P-35,126.0,NaN,FILIPOWSKI DAVID J JR,NaN,0,ROOSEVELT AVE,NaN,NaN,"PITTSBURGH, PA",15214.0,0.0,1000.0,1000.0,1000.0,0.0,0.0,1000.0,PITTSBURGH - 26TH WARD,47.0,City Of Pittsburgh,T,Taxable,10.0,Regular,R,Residential,100.0,VACANT LAND,NaN,NaN,NaN,NaN,NATIONAL CITY MTGE CO,880,ATTENTION: TAX DEPARTMENT,PO BOX 1804,"DAYTON, OH 45401-1804",NaN,04/04/95 00:00:00,40000.0,9431.0,229,4,THE FLAG CANNOT BE DETERMINED,NaN,NaN,NaN,NaN,0.0,NaN,2100.0,4661.642006,301.656075,1104177.0
470,0162P00019000000,162-P-19,126.0,NaN,MADDOX WALTER III & FRANCES,NaN,0,ROOSEVELT AVE,NaN,NaN,"PITTSBURGH, PA",15214.0,0.0,1200.0,1200.0,1200.0,0.0,0.0,1200.0,PITTSBURGH - 26TH WARD,47.0,City Of Pittsburgh,T,Taxable,10.0,Regular,R,Residential,100.0,VACANT LAND,NaN,NaN,NaN,NaN,RF,NaN,NaN,MERRIMAN RD,"SEWICKLEY, PA 15143-0000",NaN,05/05/50 00:00:00,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2500.0,2446.715218,246.522566,1103638.0
585,0040B00101000000,40-B-101,128.0,NaN,WATKINS WILLIAM C & MOLLIE G,NaN,0,HOLLYWOOD ST,NaN,NaN,"PITTSBURGH, PA",15205.0,0.0,1000.0,1000.0,1000.0,0.0,0.0,1000.0,PITTSBURGH - 28TH WARD,47.0,City Of Pittsburgh,T,Taxable,10.0,Regular,R,Residential,100.0,VACANT LAND,NaN,NaN,NaN,NaN,RF,NaN,NaN,WOODSTOCK AVE,"HOPWOOD, PA 15445-0000",NaN,05/05/50 00:00:00,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2500.0,2546.401173,254.599951,1008038.0
623,0543C00134000000,543-C-134,879.0,NaN,SCHULTZ MARY,NaN,1507,PLEASANT VIEW DR,NaN,NaN,"MONROEVILLE, PA",15146.0,0.0,3100.0,3100.0,3100.0,0.0,0.0,3100.0,MONROEVILLE,18.0,Monroeville Gateway,T,Taxable,10.0,Regular,R,Residential,100.0,VACANT LAND,NaN,NaN,NaN,NaN,UNDELIVERABLE / NO KNOWN ADDRESS,NaN,NaN,NaN,", -",NaN,05/05/50 00:00:00,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,13106.0,15999.497253,516.855660,2053794.0
655,0385B00085000000,385-B-85,870.0,NaN,MORGAN HARRY S,NaN,0,HELENA ST,NaN,NaN,"WEST MIFFLIN, PA",15122.0,0.0,15300.0,15300.0,15300.0,0.0,0.0,15300.0,WEST MIFFLIN,45.0,West Mifflin Area,T,Taxable,10.0,Regular,R,Residential,100.0,VACANT LAND,NaN,NaN,NaN,NaN,MORGAN HARRY S C/O JAMES PERICH ESQUIRE,NaN,NaN,430 BL

In [ ]:
for k in property_map.keys():
    if(len(property_map[k])>1):
        print "%s: %r" % (k,property_map[k])

In [ ]:
for k in owner_map.keys():
    if(len(owner_map[k])>1):
        print "[%d] %s: %r" % (len(owner_map[k]),k,owner_map[k])

# Process geometries to be able to record lat/lon for properties

In [395]:
#with open("assessments/Allegheny_County_Parcel_Boundaries.geojson") as f:
#    parcel_json = json.load(f)
#len(parcel_json['features'])
g = gpd.read_file('assessments/Allegheny_County_Parcel_Boundaries.geojson')
g

,OBJECTID,PIN,MAPBLOCKLOT,MUNICODE,CALCACREAGE,NOTES,PSEUDONO,MODIFIEDBY,MODIFIEDON,CREATEDBY,CREATEDON,COMMENTS,GlobalID,SHAPE_Length,SHAPE_Area,geometry
0,1,0364N00065000000,364-N-65,859,0.06,,None,t096517,11/28/2006 2:01:12 PM,None,None,None,{0FAA8146-DE79-47CC-B186-FB98F2FBFB6C},213.674930,2.646605e+03,"POLYGON ((-79.8460802965764 40.50232819790818, -79.8461027929623 40.50214073658329, -79.84624528499566 40.50215394254978, -79.84621724195085 40.50233653866371, -79.8460802965764 40.50232819790818))"
1,2,0364P00210000000,364-P-210,859,NaN,None,None,None,None,None,None,None,{D2ADA818-E86A-41A5-A9C1-E0A8EEBFC09D},287.508402,2.943190e+03,"POLYGON ((-79.84363927991781 40.50234247810071, -79.84364799755707 40.50227573638474, -79.84407420749474 40.50230456235651, -79.84406557099635 40.50237292721619, -79.84363927991781 40.50234247810071))"
2,3,0364P00316000000,364-P-316,859,NaN,None,None,None,None,None,None,None,{7DFDE3F6-30DC-407D-8817-02A0CAC5DF05},264.179839,2.821057e+03,"POLYGON ((-79.84414183816934 40.50230960726475, -79.84415082125197 40.50223693415377, -79.84452794114458 40.50226220830854, -79.84451899257283 40.50233597899358, -79.84414183816934 40.50230960726475))"
3,4,0364R00278000000,364-R-278,859,NaN,None,None,None,None,None,None,None,{5ED0E043-08AC-40A9-B76E-42DB02AEA0B9},638.612355,2.396898e+04,"POLYGON ((-79.83722721100408 40.50245920775782, -79.83686972704466 40.50231292613029, -79.83725518829219 40.50199785403606, -79.8377221079662 40.50214986999256, -79.83773211777002 40.50219723623957, -79.8377747957108 40.50232670543742, -79.83751357193492 40.50235597141347, -79.83722721100408 40...."
4,5,0364S00284000000,364-S-284,859,0.08,,None,t099136,9/28/2007 10:37:00 AM,None,None,None,{57726B72-4111-431E-9BA4-C701E46FCBDE},325.439115,3.534828e+03,"POLYGON ((-79.83406414304032 40.50244588179008, -79.8345568656643 40.50242922349889, -79.83455909786151 40.50250139611927, -79.83406809097291 40.50251539235724, -79.83406414304032 40.50244588179008))"
5,6,0364S00354000000,364-S-354,859,0.13,None,None,t099136,1/9/2007 9:45:21 AM,None,None,None,{31FCEA04-42EA-47D2-BD10-4BC27CBCA4A6},358.218826,5.872830e+03,"POLYGON ((-79.83348069515947 40.50239298906368, -79.83396774596227 40.50238197293987, -79.83397274683041 40.50249910299376, -79.83348305832195 40.50251307254067, -79.83348069515947 40.50239298906368))"
6,7,0364N00007000000,364-N-7,859,NaN,None,None,None,None,None,None,None,{ADACDAC8-C736-4662-AD16-24639AF82610},250.848595,2.680522e+03,"POLYGON ((-79.84556709815936 40.50226466605643, -79.84557511407812 40.50218985934344, -79.84592609670533 40.50221319984575, -79.84591897923639 40.50228796718578, -79.84556709815936 40.50226466605643))"
7,8,0364P00208000000,364-P-208,859,NaN,None,None,None,None,None,None,None,{3C497848-096F-4C72-91D5-5733C57575BA},291.856690,3.201590e+03,"POLYGON ((-79.84364799755707 40.50227573638474, -79.84365697477674 40.50220285757858, -79.84408362554349 40.50223043952876, -79.84407420749474 40.50230456235651, -79.84364799755707 40.50227573638474))"
8,9,0364P00318000000,364-P-318,859,NaN,None,None,None,None,None,None,None,{200A284B-B5FB-4CB5-A788-97202E610A1F},260.138379,2.608217e+03,"POLYGON ((-79.84415082125197 40.50223693415377, -79.84415941998243 40.50216735709208, -79.84453623762586 40.50219641178999, -79.84452794114458 40.50226220830854, -79.84415082125197 40.50223693415377))"
9,10,0364S00283000000,364-S-283,859,NaN,None,None,None,None,None,None,None,{428D27D0-147F-4D4F-A0A3-163968EFFEB1},376.695878,7.021928e+03,"POLYGON ((-79.8345568656643 40.50242922349889, -79.83406414304032 40.50244588179008, -79.83405967761236 40.502301468554, -79.83455263965908 40.50229258556212, -79.8345568656643 40.50242922349889))"


In [397]:
g[g.PIN=='0029G00269000000']

,OBJECTID,PIN,MAPBLOCKLOT,MUNICODE,CALCACREAGE,NOTES,PSEUDONO,MODIFIEDBY,MODIFIEDON,CREATEDBY,CREATEDON,COMMENTS,GlobalID,SHAPE_Length,SHAPE_Area,geometry
138654,138655,0029G00269000000,29-G-269,104,0.04,None,None,t099136,4/20/2005 10:51:08 AM,None,None,None,{313E42CF-6850-4067-BBF5-B4CD836EDC4E},252.657509,1741.072883,"POLYGON ((-79.95351467197403 40.43074963805737, -79.95351856121351 40.43073215077619, -79.95352391327566 40.43070809168142, -79.95391484214555 40.43076244181078, -79.95390486015087 40.43080608314526, -79.95351467197403 40.43074963805737))"


In [382]:
#data_idx = {}
#for i in range(0, len(apd_2017.index)):
#    par_id = apd_2017['PARID'][i]
#    data_idx[par_id] = i
    
#parcel_idx = {}
#for i in range(0,len(parcel_json["features"])):
#    pin = parcel_json['features'][i]['properties']['PIN']    
#    parcel_idx[pin] = i
              

In [ ]:
# This is probably slower, try the one below
centroids = {}
missed = []
for par_id in property_map.keys():
    if(len(g[g.PIN==par_id])>0):
        # Use geopandas to get the geometry for this parcel ID
        centroid = g[g.PIN==par_id]["geometry"].centroid
        # Because this used to use a direct read of the geojson and is now 
        # changed to GeoPandas, we convert the Point to the tuple that LonLatToPixelXY expects
        centroids[par_id] = (float(centroid.x),float(centroid.y))
    else:
        missed.append(par_id)


In [418]:
# This might be faster:
centroids = {}
missed = []
for i in range(0, len(g.index)):
    par_id = g['PIN'][i]
    # Use geopandas to get the geometry for this parcel ID
    centroid = g['geometry'][i].centroid
    # Because this used to use a direct read of the geojson and is now 
    # changed to GeoPandas, we convert the Point to the tuple that LonLatToPixelXY expects
    centroids[par_id] = (float(centroid.x),float(centroid.y))

In [419]:
LonLatToPixelXY(centroids['0029G00269000000'])

[71.14402403468608, 96.51512692091366]

In [420]:
print len(centroids)

578031


# New volume animation

In [390]:
def color_from_floats(r,g,b):
    return r + g * 256.0 + b * 256.0 * 256.0

def pack_color(color):
    return color['r'] + color['g'] * 256.0 + color['b'] * 256.0 * 256.0;

def parse_color(color):
    color = color.strip()
    c = color
    try:
        if c[0] == '#':
            c = c[1:]
        if len(c) == 3:
            return pack_color({'r': 17 * int(c[0:1], 16),
                               'g': 17 * int(c[1:2], 16),
                               'b': 17 * int(c[2:3], 16)})
        if len(c) == 6:
            return pack_color({'r': int(c[0:2], 16),
                               'g': int(c[2:4], 16),
                               'b': int(c[4:6], 16)})
    except:
        pass
    raise InvalidUsage('Cannot parse color <code><b>%s</b></code> from spreadsheet.<br><br>Color must be in standard web form, <code><b>#RRGGBB</b></code>, where RR, GG, and BB are each two-digit hexadecimal numbers between 00 and FF.<br><br>See <a href="https://www.w3schools.com/colors/colors_picker.asp">HTML Color Picker</a>' % color)

def parse_colors(colors):
    packed = [parse_color(color) for color in colors]
    return numpy.array(packed, dtype = numpy.float32)

In [391]:
out_suffix="_09_17"

In [422]:
start=arrow.now()

# Use the property_map, owner_map, and changeaddr_map to generate a new type of volume animation
# Write out a binary file with the volume colors
vol_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695']
def volume_to_color(volume):
    if (volume < 2):
        return parse_color(vol_colors[0]) 
    #elif (volume < 3):
    #    return parse_color(vol_colors[1]) 
    #elif (volume < 4):
    #    return parse_color(vol_colors[2]) 
    elif (volume < 5):
        return parse_color(vol_colors[3]) 
    elif (volume < 10):
        return parse_color(vol_colors[4]) 
    elif (volume < 20):
        return parse_color(vol_colors[5]) 
    elif (volume < 40):
        return parse_color(vol_colors[6]) 
    elif (volume < 60):
        return parse_color(vol_colors[7]) 
    elif (volume < 80):
        return parse_color(vol_colors[8]) 
    elif (volume < 150):
        return parse_color(vol_colors[9]) 
    elif (volume < 300):
        return parse_color(vol_colors[10]) 
    elif (volume < 500):
        return parse_color(vol_colors[11]) 
    else:
        return parse_color(vol_colors[12]) 

# Write out volume of ownership for each residential non-vacant land property
points = []
for par_id in property_map.keys():
    if(not par_id in centroids.keys()):
        print "%s is missing from centroids, skipping" % (par_id)
        continue
        
    centroid = centroids[par_id]
    # Nominally start at 1900
    next_date = '1900-01-01'

    while(next_date<this_date):
        volume_map=get_related_parids(parid, next_date)
        #print "%s: %s - %s = %d" % (parid, ret['start_date'], ret['next_date'], ret['parcount'])
        next_date=volume_map['next_date']
        color = volume_to_color(volume_map['parcount'])
        saledate = SaledateToEpoch(volume_map['start_date'])
        enddate = SaledateToEpoch(next_date)
        if(color != None):
            points += LonLatToPixelXY(centroid)        
            points.append(color)
            # Put epoch time for SALEDATE as start valid time, and next_date as end valid time
            points.append(float(saledate))
            points.append(float(enddate))
        else:
            print "Color of " + str(volume_map['parcount']) + " is None"
array.array('f', points).tofile(open(('assessments/res_volume_color_m_epoch%s.bin'%out_suffix), 'wb'))

end=arrow.now()
print "Processing took %s" % (str(end-start))


0178J00009000000 is missing from centroids, skipping
8000T01863000000 is missing from centroids, skipping
0564S00027000000 is missing from centroids, skipping
0024N00055000000 is missing from centroids, skipping
0564S00037000000 is missing from centroids, skipping
0350A00029000000 is missing from centroids, skipping
0564S00023000000 is missing from centroids, skipping
2179J00002000000 is missing from centroids, skipping
0070K00044000000 is missing from centroids, skipping
0022P00298000000 is missing from centroids, skipping
0141J00095000000 is missing from centroids, skipping
0299F00066000100 is missing from centroids, skipping
1999P00024000000 is missing from centroids, skipping
0004E00123000000 is missing from centroids, skipping
0489B00005000000 is missing from centroids, skipping
0851E00022000000 is missing from centroids, skipping
0512L00014000100 is missing from centroids, skipping
1507N00079000000 is missing from centroids, skipping
0079F00009000100 is missing from centroids, sk

1999K00018000000 is missing from centroids, skipping
0476H00150000000 is missing from centroids, skipping
0024N00054000000 is missing from centroids, skipping
0084M00364000000 is missing from centroids, skipping
1997B00118000000 is missing from centroids, skipping
0247J00157000000 is missing from centroids, skipping
0461A00043000000 is missing from centroids, skipping
0464L00135000000 is missing from centroids, skipping
8000T03070000000 is missing from centroids, skipping
1735P00025000000 is missing from centroids, skipping
0851E00020000000 is missing from centroids, skipping
0023M00247000000 is missing from centroids, skipping
0131E00042000000 is missing from centroids, skipping
0050K00060000000 is missing from centroids, skipping
1202D00218000000 is missing from centroids, skipping
1202H00200000000 is missing from centroids, skipping
0851E00028000000 is missing from centroids, skipping
8000T00347000000 is missing from centroids, skipping
0119S00159000A00 is missing from centroids, sk

0050K00052000000 is missing from centroids, skipping
0086N00185000200 is missing from centroids, skipping
1658M00433000000 is missing from centroids, skipping
0034K00043000000 is missing from centroids, skipping
1658M00026000000 is missing from centroids, skipping
9946X50494000000 is missing from centroids, skipping
0042B00020000000 is missing from centroids, skipping
0080H00007000000 is missing from centroids, skipping
0564S00031000000 is missing from centroids, skipping
0049R00220000000 is missing from centroids, skipping
9946X83322000000 is missing from centroids, skipping
0326D00012000000 is missing from centroids, skipping
0738M00192000000 is missing from centroids, skipping
0506N00003000000 is missing from centroids, skipping
0084M00336000000 is missing from centroids, skipping
9908X82945000000 is missing from centroids, skipping
0382G00202000000 is missing from centroids, skipping
0119S00103000100 is missing from centroids, skipping
1999K00014000000 is missing from centroids, sk

KeyboardInterrupt: 

In [ ]:
# Below is Anne's attempt to put floating point representations of colors into the .bin file for a new 
# shader Randy's writing

In [ ]:
def incrementKey(adict,key):
    if(key == '' or math.isnan(key)):
       return
    if not adict.has_key(key):
        adict[key] = 1
    else:
        adict[key] += 1

In [ ]:
year_built_colors = ['#ffffe0','#ffdaa3','#ffb27c','#fb8768','#eb5f5b','#d3394a','#b3152f','#8b0000']
def year_built_to_color(year):
    if (year <= 1900.0):
        return color_from_floats(64,64,64)
    elif (year < 1916.0):
        return parse_color(year_built_colors[7])       
    elif (year < 1932.0):
        return parse_color(year_built_colors[6])
    elif (year < 1948.0):
        return parse_color(year_built_colors[5])
    elif (year < 1964.0):
        return parse_color(year_built_colors[4])
    elif (year < 1980.0):
        return parse_color(year_built_colors[3])
    elif (year < 1996.0):
        return parse_color(year_built_colors[2])
    elif  (year < 2012.0):
        return parse_color(year_built_colors[1])
    elif  (year < 2016.0):
        return parse_color(year_built_colors[0])
    else:
        return color_from_floats(255,255,255)
 
   
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(year_built_to_color(float(year_built)))
        # Set start valid time as year_built, and max positive float as end valid time
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(float(1e38))
array.array('f', points).tofile(open(('assessments/parcels_bltyr_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
# Hightlight based on likelyhood of lead
# <1950 = Probably lead
# 1950-1978 = Maybe lead (not illegal, but being phased out)
# >1978 = Probably not lead (illegal)
year_built_colors = ['#ffffe0','#ffdaa3','#ffb27c','#fb8768','#eb5f5b','#d3394a','#b3152f','#8b0000']
def year_built_to_lead_color(year):
    if (year <= 1900.0):
        return color_from_floats(64,64,64)
    elif (year < 1950.0):
        return parse_color(year_built_colors[7])       
    elif (year < 1979.0):
        return parse_color(year_built_colors[3])
    else:
        return parse_color(year_built_colors[0])
    
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(year_built_to_lead_color(float(year_built)))
        # Set start valid time as year_built, and max positive float as end valid time
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(float(1e38))
array.array('f', points).tofile(open(('assessments/parcels_bltyr_lead_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
                                     
fm_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695']
def fm_total_to_color(fm_total):
    if (fm_total < 25000.):
        return parse_color(fm_colors[0]) 
    elif (fm_total < 50000.):
        return parse_color(fm_colors[1]) 
    elif (fm_total < 75000.):
        return parse_color(fm_colors[2]) 
    elif (fm_total < 100000.):
        return parse_color(fm_colors[3]) 
    elif (fm_total < 125000.):
        return parse_color(fm_colors[4]) 
    elif (fm_total < 150000.):
        return parse_color(fm_colors[5]) 
    elif (fm_total < 200000.):
        return parse_color(fm_colors[6]) 
    elif (fm_total < 250000.):
        return parse_color(fm_colors[7]) 
    elif (fm_total < 300000.):
        return parse_color(fm_colors[8]) 
    elif (fm_total < 400000.):
        return parse_color(fm_colors[9]) 
    elif (fm_total < 500000.):
        return parse_color(fm_colors[10]) 
    elif (fm_total < 750000.):
        return parse_color(fm_colors[11]) 
    else:
        return parse_color(fm_colors[12]) 
 
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    fm_total = float(apd['FAIRMARKETTOTAL'][i])   
    
    year_built = apd['YEARBLT'][i]
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)

    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(fm_total_to_color(fm_total))
        # Set start valid time as year_built, and max positive float as end valid time
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(float(1e38))
array.array('f', points).tofile(open(('assessments/parcels_fm_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']
class_descriptions = sorted(class_descs.keys())

def class_desc_to_color(class_description):
    index = class_descriptions.index(class_description)
    return(parse_color(parcel_colors[index]))

points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    class_description = apd['CLASSDESC'][i] 
    year_built = apd['YEARBLT'][i]

    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)

    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(class_desc_to_color(class_description))
        # Set start valid time as year_built, and max positive float as end valid time
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(float(1e38))
array.array('f', points).tofile(open(('assessments/parcels_class_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
# Highlight vacant lots
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    class_description = apd['CLASSDESC'][i] 
    usedesc = apd['USEDESC'][i]

    # default to grey
    color = "#555555"
    if(usedesc =='VACANT LAND'):
        # vacant lot, make it green
        color = "#02ca75"

    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(parse_color(color))
        # Set start valid time as forever, and max positive float as end valid time
        points.append(0)
        points.append(float(1e38))
array.array('f', points).tofile(open(('assessments/vacant_lots%s.bin'%out_suffix), 'wb'))

In [ ]:
# Here are Anne's efforts using the version with owner names to generate volume of properties owned by current owner

In [ ]:
# SALEDATE field is in '%m-%d-%Y' format (ex 10-26-2012) when present
def SaledateToEpoch(datestr):
    return calendar.timegm(time.strptime(datestr, '%m-%d-%Y'))

In [ ]:
# Parse apd table into dictionaries from owner names and change addresses to parid, and from parid to other info
# about the property
owner_names_to_parids = {}
owner_changeaddrs_to_parids = {}
parid_info = {}
volume_counts = {}

# For now, only look at residential properties with known centroids in 15213
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    zipcode = apd['PROPERTYZIP'][i]
    class_description = apd['CLASSDESC'][i] 
    usedesc = apd['USEDESC'][i] 
    if (usedesc == 'VACANT LAND' or not centroids.has_key(par_id)):
        # skip this one
        continue
    
    # We want to include residential, apartments (usedesc includes APART)
    if(class_description != 'RESIDENTIAL' and (isinstance(usedesc, numbers.Number) or 'APART' not in usedesc)):
        # skip this one
        continue
        
    # This one meets our criteria
    owner_name = apd['PROPERTYOWNER'][i]
    owner_changeaddr = string.strip(str(apd['CHANGENOTICEADDRESS1'][i]))
    c2 = string.strip(str(apd['CHANGENOTICEADDRESS2'][i]))
    c3 = string.strip(str(apd['CHANGENOTICEADDRESS3'][i]))
    c4 = apd['CHANGENOTICEADDRESS4'][i]
    if(c2 != ''):
        owner_changeaddr+=", " + c2
    if(c3 != ''):
        owner_changeaddr+=", " + c3
    if(c4 != ''):
        if(isinstance(c4, numbers.Number)):
            if(not math.isnan(float(c4))):
                owner_changeaddr+=", " + str(int(c4))
        else:
            owner_changeaddr+=", " + c4
        
    if(owner_changeaddr == ''):
        # skip this one
        continue
        
    if(owner_names_to_parids.has_key(owner_name)):
        owner_names_to_parids[owner_name].append(par_id)
    else:
        owner_names_to_parids[owner_name]=[par_id]
        
    if(owner_changeaddrs_to_parids.has_key(owner_changeaddr)):
        owner_changeaddrs_to_parids[owner_changeaddr].append(par_id)
    else:
        owner_changeaddrs_to_parids[owner_changeaddr]=[par_id]

    parid_info[par_id]={'owner_name':owner_name,
                        'owner_changeaddr':owner_changeaddr,
                        'centroid':centroids[par_id],
                        'HOMESTEADFLAG':apd['HOMESTEADFLAG'][i],
                        'USEDESC':apd['USEDESC'][i],
                        'OWNERDESC':apd['OWNERDESC'][i],
                        'SALEDATE':apd['SALEDATE'][i],
                        'OWNERDESC':apd['OWNERDESC'][i],
                        'CLASSDESC':apd['CLASSDESC'][i]
                       }

In [ ]:
# Add counts to parid_info based on max of owner_names_to_parids or owner_changeaddrs_to_parids
# Special case properties with HOMESTEADFLAG set to HOM to be volume=1
for par_id in parid_info.keys():
    volume=1
    if(parid_info[par_id]['HOMESTEADFLAG']!='HOM'):
        ownername_num = len(owner_names_to_parids[parid_info[par_id]['owner_name']])
        ownerchangeaddr_num = len(owner_changeaddrs_to_parids[parid_info[par_id]['owner_changeaddr']])
        volume = max(ownername_num,ownerchangeaddr_num)
    parid_info[par_id]['volume'] = volume
    incrementKey(volume_counts,volume)

In [ ]:
volume_counts

In [ ]:
for ownername in owner_names_to_parids.keys():
    ownername_num = len(owner_names_to_parids[ownername])
    if(ownername_num>10):
        print "%s: %d" % (ownername,ownername_num)

In [ ]:
for ownerchangeaddr in owner_changeaddrs_to_parids.keys():
    ownerchangeaddr_num = len(owner_changeaddrs_to_parids[ownerchangeaddr])
    if(ownerchangeaddr_num>10):
        print "%s: %d" % (ownerchangeaddr,ownerchangeaddr_num)

In [ ]:
for par_id in parid_info.keys():
    if(parid_info[par_id]['volume']>10):
        print parid_info[par_id]

In [ ]:
# Write out a binary file with the volume colors
vol_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695']
def volume_to_color(volume):
    if (volume < 2):
        return parse_color(vol_colors[0]) 
    #elif (volume < 3):
    #    return parse_color(vol_colors[1]) 
    #elif (volume < 4):
    #    return parse_color(vol_colors[2]) 
    elif (volume < 5):
        return parse_color(vol_colors[3]) 
    elif (volume < 10):
        return parse_color(vol_colors[4]) 
    elif (volume < 20):
        return parse_color(vol_colors[5]) 
    elif (volume < 40):
        return parse_color(vol_colors[6]) 
    elif (volume < 60):
        return parse_color(vol_colors[7]) 
    elif (volume < 80):
        return parse_color(vol_colors[8]) 
    elif (volume < 150):
        return parse_color(vol_colors[9]) 
    elif (volume < 300):
        return parse_color(vol_colors[10]) 
    elif (volume < 500):
        return parse_color(vol_colors[11]) 
    else:
        return parse_color(vol_colors[12]) 

# Write out volume of ownership for each residential non-vacant land property
points = []
for par_id in parid_info.keys():
    centroid = parid_info[par_id]['centroid']
    color = volume_to_color(parid_info[par_id]['volume'])
    saledate = SaledateToEpoch('01-01-1900')
    saledate_raw = parid_info[par_id]['SALEDATE']
    if(not(saledate_raw == '' or (isinstance(saledate_raw, numbers.Number) and math.isnan(saledate_raw)))):
        # If not valid, leave as 1900, otherwise parse it into an epoch time
        saledate = SaledateToEpoch(saledate_raw)
        
    if(color != None):
        points += LonLatToPixelXY(centroid)        
        points.append(color)
        # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
        points.append(float(saledate))
        points.append(float(1e38))
    else:
        print "Color of " + str(parid_info[par_id]['volume']) + " is None"
array.array('f', points).tofile(open(('assessments/res_volume_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
#original version
legend = '<svg width="400" height="30"><rect fill="#a50026" x="0" height="10" width="30"></rect><rect fill="#cd2827" x="30" height="10" width="30"></rect><rect fill="#e75436" x="60" height="10" width="30"></rect><rect fill="#f7804b" x="90" height="10" width="30"></rect><rect fill="#fdad61" x="120" height="10" width="30"></rect><rect fill="#fed788" x="150" height="10" width="30"></rect><rect fill="#ffffbf" x="180" height="10" width="30"></rect><rect fill="#b9e0ed" x="210" height="10" width="30"></rect><rect fill="#8dc0db" x="240" height="10" width="30"></rect><rect fill="#699fca" x="270" height="10" width="30"></rect><rect fill="#4d7db9" x="300" height="10" width="30"></rect><rect fill="#3e5aa7" x="330" height="10" width="30"></rect><text font-size="10.5px" y="29" x="0" fill="#ffffff">1</text><text font-size="10.5px" y="29" x="20"  fill="#ffffff">&nbsp;2</text><text font-size="10.5px" y="29" x="50"  fill="#ffffff">&nbsp;3</text><text font-size="10.5px" y="29" x="80"  fill="#ffffff"> 4</text><text font-size="10.5px" y="29" x="110"  fill="#ffffff"> 5</text><text font-size="10.5px" y="29" x="140"  fill="#ffffff">10</text><text font-size="10.5px" y="29" x="170"  fill="#ffffff">20</text><text font-size="10.5px" y="29" x="200"  fill="#ffffff">40</text><text font-size="10.5px" y="29" x="230"  fill="#ffffff">60</text><text font-size="10.5px" y="29" x="260"  fill="#ffffff">80</text><text font-size="10.5px" y="29" x="290"  fill="#ffffff">150</text><text font-size="10.5px" y="29" x="320"  fill="#ffffff">300</text><text font-size="10.5px" y="29" x="350"  fill="#ffffff">500</text>'
HTML(legend)

In [ ]:
#modified version
legend = '<svg width="400" height="30"><rect fill="#a50026" x="0" height="10" width="30"></rect><rect fill="#f7804b" x="30"  height="10" width="30"></rect><rect fill="#fdad61" x="60"  height="10" width="30"></rect><rect fill="#fed788" x="90"  height="10" width="30"></rect><rect fill="#ffffbf" x="120" height="10" width="30"></rect><rect fill="#b9e0ed" x="150" height="10" width="30"></rect><rect fill="#8dc0db" x="180" height="10" width="30"></rect><rect fill="#699fca" x="210" height="10" width="30"></rect><rect fill="#4d7db9" x="240" height="10" width="30"></rect><rect fill="#3e5aa7" x="270" height="10" width="30"></rect><text font-size="10.5px" y="29" x="0" fill="#ffffff">1</text><text font-size="10.5px" y="29" x="20"  fill="#ffffff">&nbsp;2</text><text Font-size="10.5px" y="29" x="50"   fill="#ffffff"> 5</text><text font-size="10.5px" y="29" x="80"   fill="#ffffff">10</text><text font-size="10.5px" y="29" x="110"  fill="#ffffff">20</text><text font-size="10.5px" y="29" x="140"  fill="#ffffff">40</text><text font-size="10.5px" y="29" x="170"  fill="#ffffff">60</text><text font-size="10.5px" y="29" x="200"  fill="#ffffff">80</text><text font-size="10.5px" y="29" x="230"  fill="#ffffff">150</text><text font-size="10.5px" y="29" x="260"  fill="#ffffff">300</text><text font-size="10.5px" y="29" x="290"  fill="#ffffff">500</text>'
HTML(legend)

In [ ]:
# Write out a binary file with the colors for residential owned by corporate, government, or regular owners
def ownertype_to_color(par_id):
    ownerdesc = parid_info[par_id]['OWNERDESC']
    classdesc = parid_info[par_id]['CLASSDESC']
    volume = parid_info[par_id]['volume']
    if (classdesc == "GOVERNMENT"):
        print "Government: %s, %s, %d" % (ownerdesc, classdesc,volume)
        # Same color as government in property class view
        return parse_color("#d1947a") 
    elif ('CORPORATION' in ownerdesc):
        # Same color as 500+ properties in volume view
        #return parse_color('#313695') 
        return parse_color('#ffffbf')
    elif ('REGULAR' in ownerdesc):
        # Regular owner, what volume?
        if(volume == 1):
            # Same color as 1 in volume view
            return parse_color('#a50026')
        else:
            # Same color as other in class view
            return parse_color('#02ca75')
    else:
        print "Unrecognized owner type: %s, %s, %d" % (ownerdesc, classdesc,volume)
        return parse_color('#303030') 

# Write out volume of ownership for each residential non-vacant land property
points = []
for par_id in parid_info.keys():
    centroid = parid_info[par_id]['centroid']
    color = ownertype_to_color(par_id)
    saledate = SaledateToEpoch('01-01-1900')
    saledate_raw = parid_info[par_id]['SALEDATE']
    if(not(saledate_raw == '' or (isinstance(saledate_raw, numbers.Number) and math.isnan(saledate_raw)))):
        # If not valid, leave as 1900, otherwise parse it into an epoch time
        saledate = SaledateToEpoch(saledate_raw)
        
    if(color != None):
        points += LonLatToPixelXY(centroid)        
        points.append(color)
        # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
        points.append(float(saledate))
        points.append(float(1e38))
    else:
        print "Color of " + str(parid_info[par_id]['volume']) + " is None"
array.array('f', points).tofile(open(('assessments/ownertype_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
parid_info['0002K00028000000']

In [ ]:
# Foreclosures

In [ ]:
# SALEDATE field is in '%m-%d-%Y' format (ex 10-26-2012) when present
def FilingdateToEpoch(datestr):
    return calendar.timegm(time.strptime(datestr, '%Y-%m-%d'))

In [ ]:
# Read in foreclosure data 
fpath = "assessments/foreclosures-180327.csv"
fapd = pandas.read_csv(fpath)

In [ ]:
# Write out volume of ownership for each residential non-vacant land property
points = []
missing_pin_c=0
found_pin_c=0
missing_pin_a=0
found_pin_a=0
foreclosure_color = '#ff00ff'

for i in range(0, len(fapd.index)):
    par_id = fapd['pin'][i]
    filing_date_raw = fapd['filing_date'][i]
    
    if(filing_date_raw == '' or (isinstance(filing_date_raw, numbers.Number) and math.isnan(filing_date_raw))):
        print "%d: Can't read filing_date %s" % (i, filing_date_raw)
        continue
    filing_date = FilingdateToEpoch(filing_date_raw)
    
    if par_id not in parid_info:
        missing_pin_c+=1
        continue
    found_pin_c+= 1
    
    centroid = parid_info[par_id]['centroid']
    
    if par_id not in parid_info:
        missing_pin_a+=1
    found_pin_a+=1
    
    saledate = SaledateToEpoch('01-01-1900')
    saledate_raw = parid_info[par_id]['SALEDATE']
    if(not(saledate_raw == '' or (isinstance(saledate_raw, numbers.Number) and math.isnan(saledate_raw)))):
        # If not valid, leave as 1900, otherwise parse it into an epoch time
        saledate = SaledateToEpoch(saledate_raw)
        
    startdate = filing_date
    enddate = saledate
    
    if(filing_date<saledate):
        # Sold after foreclosure, who bought it?
        points += LonLatToPixelXY(centroid)        
        points.append(ownertype_to_color(par_id))
        # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
        points.append(float(filing_date))
        points.append(float(1e38))
    else:
        # Sold before foreclosure, who owned it before?
        points += LonLatToPixelXY(centroid)        
        points.append(ownertype_to_color(par_id))
        # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
        points.append(float(saledate))
        points.append(float(filing_date))
        # Not sold after foreclosure, show ownership before sale and end foreclosure at end of time
        enddate=float(1e38)
        
    # Output foreclosure dot
    points += LonLatToPixelXY(centroid)        
    points.append(parse_color(foreclosure_color))
    # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
    points.append(float(startdate))
    points.append(float(enddate))

array.array('f', points).tofile(open(('assessments/foreclosure_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
# Below here are modified versions of the blocks from Gabriel's original python notebook,
# none of which are needed anymore

In [ ]:
sale_prices = {}
for i in range(0, len(apd.index)):
    sale_price = apd['SALEPRICE'][i]
    if not sale_prices.has_key(sale_price):
        sale_prices[sale_price] = 1
    else:
        sale_prices[sale_price] += 1


In [ ]:
len(sale_prices.keys())


In [ ]:
years_built = {}
sale_dates = {}
fairmarket_totals = {}
classes = {}
class_descs = {}

def incrementKey(adict,key):
    if(key == '' or math.isnan(key)):
       return
    if not adict.has_key(key):
        adict[key] = 1
    else:
        adict[key] += 1
    
for i in range(0, len(apd.index)):
    year_built = apd['YEARBLT'][i]
    incrementKey(years_built, year_built)
    sale_date = apd['SALEDATE'][i]
    incrementKey(sale_dates, sale_date)
    fairmarket_total = apd['FAIRMARKETTOTAL'][i]
    incrementKey(fairmarket_totals, fairmarket_total)
    class_ = apd['CLASS'][i]
    incrementKey(classes, class_)
    class_desc = apd['CLASSDESC'][i]
    incrementKey(class_descs, class_desc)


In [ ]:
i

In [ ]:
len(apd.index)

In [ ]:
len(years_built.keys())
years_built['']

In [ ]:
years_built

In [ ]:
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
array.array('f', points).tofile(open(('assessments/parcels_yr%s.bin'%out_suffix), 'wb'))


In [ ]:
year_built
date = datetime.datetime.strptime('2000', '%Y')   
GetEpoch(date)

In [ ]:
fairmarket_totals['0']

In [ ]:
classes

In [ ]:
class_descs

In [ ]:
sorted(class_descs.keys())

In [ ]:
no_year_built = 0
no_= 0
for i in range(0, len(apd.index)):
    year_built = apd['YEARBLT'][i]
    class_ = apd['CLASS'][i]
    if year_built == '' and class_ == 'R':
        no_year_built += 1
    elif year_built == '':
        no_ += 1
print no_year_built
print no_

In [ ]:
class_descriptions = sorted(class_descs.keys())
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    class_description = apd['CLASSDESC'][i]  
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(class_descriptions.index(class_description))
array.array('f', points).tofile(open(('assessments/parcels_yrblt%s.bin'%out_suffix), 'wb'))


In [ ]:
foo = sorted(class_descs.keys())

In [ ]:
len(foo)

In [ ]:
parcel_colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628']

In [ ]:
for color in hex_colors:
    rgb_color = HexToRgb(color)
    print "%s,%s,%s" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0) 

In [ ]:
hex_colors = ["#fb3059",
"#fe6b2d",
"#d1947a",
"#c6a900",
"#02ca75",
"#00a2de",
"#9529b1"]
for color in hex_colors:
    rgb_color = HexToRgb(color)
    print "%.4f,%.4f,%.4f" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0) 

In [ ]:
hex_colors = ['#ffffff','#fffffa','#fffdf5','#fffdf1','#fffcef','#fffbed','#fffbea','#fff9e6','#fff9e3','#fff7e0','#fff7dd','#fff6db','#fff5d8','#fff4d6','#fff3d4','#fff3d2','#fff2d0','#fff0cd','#fff0cb','#ffefc8','#ffeec6','#ffedc3','#ffecc1','#ffeabe','#ffeabc','#ffe9b9','#ffe7b7','#ffe7b4','#ffe6b2','#ffe4b0','#ffe4ad','#ffe3ab','#ffe1a8','#ffe1a6','#ffe0a4','#ffdea1','#ffde9f','#ffdc9d','#ffdc9b','#ffda98','#ffd996','#ffd994','#ffd891','#ffd68f','#ffd68d','#ffd58a','#ffd388','#ffd386','#ffd284','#ffd081','#ffcf7f','#ffcf7d','#ffcd7b','#ffcc78','#ffcc76','#ffca74','#ffc971','#ffc96f','#ffc76d','#ffc76b','#ffc568','#ffc566','#ffc464','#ffc361','#ffc15d','#ffc05a','#ffbf58','#ffbe56','#ffbc53','#ffbc51','#ffba4e','#ffba4c','#ffb849','#ffb747','#ffb644','#ffb53f','#ffb43d','#ffb23a','#ffb237','#ffb134','#ffaf31','#ffaf2e','#ffad2a','#ffab24','#ffab21','#ffa91c','#ffa818','#ffa813','#ffa60b','#ffa400','#ffa300','#ffa200','#ffa100','#ff9f00','#ff9e00','#ff9d00','#ff9c00','#ff9b00','#ff9900','#ff9800','#ff9700','#ff9500','#ff9500','#ff9300','#ff9100','#ff9000','#ff8f00','#ff8d00','#ff8c00','#ff8a00','#ff8a00','#fe8800','#fe8800','#fe8700','#fe8400','#fd8400','#fd8300','#fd8200','#fd8000','#fc7f00','#fc7d00','#fc7c00','#fc7b00','#fb7a00','#fb7900','#fb7800','#fa7700','#fa7400','#fa7400','#f97200','#f97100','#f87000','#f86e00','#f86e00','#f76d00','#f76c00','#f66a00','#f66900','#f56800','#f56600','#f46600','#f46400','#f36400','#f36200','#f36100','#f26000','#f15e00','#f15d00','#f05d00','#ef5b00','#ef5900','#ef5900','#ee5800','#ed5700','#ed5500','#ec5500','#eb5400','#ea5200','#ea5100','#e95000','#e84f00','#e84e00','#e74c00','#e74b00','#e64a00','#e54900','#e44800','#e44700','#e34600','#e34500','#e14400','#e14300','#e04100','#df4000','#de4000','#de3e00','#dd3d00','#dc3d00','#db3c00','#db3a00','#da3900','#d83800','#d83700','#d73600','#d63500','#d63401','#d43301','#d43201','#d23101','#d13001','#d12e01','#d02e01','#d02d01','#ce2c01','#ce2b01','#cd2a01','#cc2901','#ca2801','#c92701','#c92601','#c82501','#c72401','#c62301','#c52201','#c42102','#c32002','#c21f02','#c01e02','#bf1d02','#bf1c02','#be1b02','#bd1a02','#bc1902','#ba1902','#ba1802','#b81702','#b81602','#b71502','#b51402','#b51302','#b41202','#b21202','#b11102','#b01002','#af0e02','#ae0d02','#ac0d02','#ac0c02','#aa0b02','#a90a02','#a80902','#a80902','#a60802','#a50702','#a30602','#a30602','#a20502','#a00502','#a00402','#9e0302','#9c0302','#9b0302','#9b0201','#9a0201','#990201','#970101','#950101','#950101','#940101','#920001','#920001','#900001','#8f0000','#8d0000','#8d0000','#8b0000']

img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(img.size[0]):    # for every pixel:
    for j in range(img.size[1]):
        pixels[i,j] = hex_to_rgb(hex_colors[i]) # set the colour accordingly
img.show
img.save("assessments/year-built-color-map.png")


In [ ]:
hex_stops =  ['#1d1e4e','#482045','#6f1f4b','#8d2747','#aa3f4c','#a9513f','#ba663a','#cf8139','#e59f44','#e9bb76']
hex_colors = ['#1d1e4e','#201e4e','#241e4d','#261e4d','#291e4c','#291f4c','#2b1f4c','#2d1f4b','#301f4b','#331f4a','#341f4a','#371f49','#381f49','#3b1f48','#3d1f48','#3d1f48','#3f2047','#402047','#432046','#452046','#462046','#482045','#492045','#4b2045','#4d2046','#4d2046','#502046','#502046','#522047','#532047','#552047','#562047','#572047','#5a2048','#5b2048','#5c2048','#5d2048','#5e2048','#602049','#622049','#632049','#652049','#65204a','#67204a','#69204a','#6b1f4a','#6b1f4a','#6c1f4b','#6f1f4b','#701f4b','#711f4b','#72204b','#72204b','#75204a','#75214a','#76214a','#77214a','#78214a','#7a224a','#7a224a','#7d2249','#7d2249','#7e2349','#7f2349','#802449','#812449','#832448','#842548','#852548','#862548','#882548','#882648','#892648','#8b2647','#8b2747','#8d2747','#8e2847','#8e2847','#902947','#902948','#912a48','#922b48','#932c48','#932c48','#952d48','#952d48','#952e49','#972f49','#973049','#983049','#993149','#9a3249','#9a3249','#9c334a','#9c334a','#9d354a','#9d354a','#9f364a','#9f364a','#a0374a','#a1384a','#a2384b','#a2394b','#a33a4b','#a43a4b','#a43b4b','#a63c4b','#a73c4b','#a83d4c','#a83d4c','#a93e4c','#a93f4c','#aa414b','#aa424a','#aa4349','#aa4448','#aa4747','#aa4846','#a94b44','#a94b43','#a94d43','#a94e41','#a94f40','#a9513f','#a9523f','#aa523f','#ab533e','#ac543e','#ac543e','#ad563e','#ad563e','#ae573e','#ae583e','#af593d','#b05a3d','#b05a3d','#b15b3d','#b15b3d','#b25d3c','#b35d3c','#b35e3c','#b45f3c','#b55f3c','#b5603c','#b6623b','#b6623b','#b7623b','#b8643b','#b9643b','#b9653a','#ba653a','#ba673a','#bb683a','#bb683a','#bc683a','#bd693a','#bd6a3a','#bd6a3a','#be6c3a','#bf6c3a','#c06d3a','#c16e3a','#c16e3a','#c16f3a','#c2713a','#c2713a','#c3723a','#c4733a','#c5743a','#c5753a','#c5753a','#c6753a','#c7773a','#c8773a','#c87939','#c97939','#c97939','#ca7a39','#ca7b39','#cb7c39','#cc7d39','#cc7e39','#cd7e39','#ce7f39','#ce8039','#cf8139','#cf8139','#d08239','#d0833a','#d1843a','#d2843a','#d2853b','#d2863b','#d3873b','#d4883b','#d5883c','#d5893c','#d58a3c','#d68b3c','#d78c3d','#d78d3d','#d88d3d','#d88d3e','#d98f3e','#da8f3e','#da903f','#da913f','#db923f','#dc933f','#dc9340','#dd9340','#dd9440','#de9641','#df9641','#df9741','#e09741','#e09942','#e19942','#e19a42','#e29a42','#e29c43','#e39d43','#e49d43','#e59e44','#e59e44','#e59f45','#e5a047','#e6a149','#e6a24b','#e6a44c','#e6a54e','#e6a650','#e6a652','#e6a652','#e7a754','#e7a855','#e7aa57','#e7ab59','#e7ab5b','#e7ac5d','#e8ae5e','#e8af60','#e8af62','#e8b063','#e8b165','#e8b265','#e8b367','#e8b469','#e8b56b','#e9b66c','#e9b76e','#e9b870','#e9b872','#e9ba73','#e9ba75','#e9bb76']

In [ ]:
hex_colors.index(hex_stops[9])

In [ ]:
years_built

In [ ]:
hex_stops = ['#f7fbff','#deebf7','#c6dbef','#9ecae1','#6baed6','#4292c6','#2171b5','#08519c','#08306b']

In [ ]:
for i in hex_stops:
  print i + ","

In [ ]:
hex_stops = ['#ffffe0','#dcfac5','#b8f4ab','#90ee90','#6dc88c','#46a386','#008080','#265a81','#233381','#000080']
for color in hex_stops:
    rgb_color = HexToRgb(color)
    print "%.4f,%.4f,%.4f" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0) 

In [ ]:
hex_stops = ['#1d1e4e','#5f2049','#8c2747','#aa414b','#ba653a','#d0823a','#e5a045','#ebc07f','#f7e0af','#ffffe0']
years = [1880,1895,1910,1925,1940,1955,1970,1985,2000,2015]

In [ ]:
for i in range(0,10):
    print "if (year < %s) {" % (years[i])
    rgb_color = HexToRgb(hex_stops[i])
    print "  color = vec4(%.4f,%.4f,%.4f,1.0)" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0)
    print "}"

In [ ]:
len(years)

In [ ]:
hex_colors = ['#ffffe0','#ffeec1','#ffdea7','#ffcb91','#ffb880','#ffa474','#fe906a','#f87d64','#f06a5e','#e75758','#db4551','#cf3447','#c0223b','#b0122c','#9e051b','#8b0000']
img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(img.size[0]):    # for every pixel:
    for j in range(img.size[1]):
        pixels[i,j] = hex_to_rgb(hex_colors[i/16]) # set the colour accordingly
img.show()
img.save("assessments/year-built-color-map.png")



In [ ]:
parcel_colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628']
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']

year_built_colors = ['#ffffe0','#ffeec1','#ffdea7','#ffcb91','#ffb880','#ffa474','#fe906a','#f87d64','#f06a5e','#e75758','#db4551','#cf3447','#c0223b','#b0122c','#9e051b','#8b0000']
year_built_colors = ['#ffffe0','#ffdaa3','#ffb27c','#fb8768','#eb5f5b','#d3394a','#b3152f','#8b0000']
img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(0,len(parcel_colors)):
    pixels[0,i] = HexToRgb(parcel_colors[i]) # set the colour accordingly
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    pixels[1,j] = HexToRgb(year_built_colors[i]) # set the colour accordingly
    j += 1
#img.show()
img.save("assessments/color-map.png")


In [ ]:
class_descriptions = sorted(class_descs.keys())
points = []
array.array('f', points).tofile(open('parcels_class.bin', 'wb'))
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    class_description = apd['CLASSDESC'][i]   
    if year_built == '' or year_built == '0':
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(float(year_built))
        points.append(class_descriptions.index(class_description)+1.0)
array.array('f', points).tofile(open('assessments/parcels_class.bin', 'wb'))


In [ ]:
sorted(class_descs.keys())

In [ ]:
class_descriptions.index('RESIDENTIAL')

In [ ]:
points[10000:10005]

In [ ]:
def scale_year(year):
    if year < 1800.0: 
        year = 1800.0
    return int(((year - 1800.0) * 8.0) / (2016.0-1800.))


In [ ]:
scale_year(1800.0)

In [ ]:
len(year_built_colors)

In [ ]:
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    print '<rect fill="%s" x="%s" height="10" width="25" stroke-width="1px" stroke="#666"></rect>' % (year_built_colors[i],j*25)
    j += 1
    
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    print '<text font-size="10.5px" fill="%s" y="29" x="%s">%s</text>' % (year_built_colors[i], j*25, 1800)
    j += 1


In [ ]:
list(reversed(range(10)))

In [ ]:
len(fairmarket_totals)

In [ ]:
fm_keys = sorted(map(int,fairmarket_totals.keys()))

In [ ]:
numpy.average(fm_keys)

In [ ]:
numpy.mean(fm_keys)

In [ ]:
numpy.std(fm_keys)

In [ ]:
fairmarket_totals['50000']

In [ ]:
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']
year_built_colors = ['#ffffe0','#ffdaa3','#ffb27c','#fb8768','#eb5f5b','#d3394a','#b3152f','#8b0000']
fm_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695'
]

img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(0,len(parcel_colors)):
    pixels[0,i] = HexToRgb(parcel_colors[i]) # set the colour accordingly
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    pixels[1,j] = HexToRgb(year_built_colors[i]) # set the colour accordingly
    j += 1
for i in range(0,len(fm_colors)):
    pixels[2,i] = HexToRgb(fm_colors[i]) # set the colour accordingly
    
#img.show()
img.save("assessments/color-map.png")


In [ ]:
class_descriptions = sorted(class_descs.keys())
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    class_description = apd['CLASSDESC'][i] 
    fm_total = float(apd['FAIRMARKETTOTAL'][i])    
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(float(year_built))
        points.append(class_descriptions.index(class_description)+1.0)
        points.append(fm_total)
array.array('f', points).tofile(open(('assessments/parcels_all%s.bin'%out_suffix), 'wb'))


In [ ]:
sorted(class_descs.keys())

In [ ]:
# Below is Anne's attempt to put floating point representations of colors into the .bin file for a new 
# shader Randy's writing

In [ ]:
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']
class_descriptions = sorted(class_descs.keys())

def class_desc_to_color(class_description):
    index = class_descriptions.index(class_description)
    return(parse_color(parcel_colors[index]))

points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    class_description = apd['CLASSDESC'][i] 
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(class_desc_to_color(class_description))
array.array('f', points).tofile(open(('assessments/parcels_class_color%s.bin'%out_suffix), 'wb'))

In [ ]:
len(fm_colors)

In [ ]:
fairmarket_totals['0']

In [ ]:
plt.plot(map(int,fairmarket_totals.keys()), fairmarket_totals.values(), 'ro')
plt.show()

In [ ]:
fm_keys[len(fm_keys) - 100:len(fm_keys)]

In [ ]:
fm_values = ["0", "25K", "50K", "75K", "100K", "125K", "150K", "200K", "250K", "300K", "400K", "500K", "+500K"]
for i in range(len(fm_colors)):
    print '<rect fill="%s" x="%s" height="10" width="30"></rect>' % (fm_colors[i],i*30)
    
for i in range(len(fm_values)):
    print '<text font-size="10.5px" y="29" x="%s">%s</text>' % (i*30, fm_values[i])
